<a href="https://colab.research.google.com/github/ljqljqljq8/0115_Git_Build/blob/main/Type_One_Fine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Label 解析 / 不同类型样本示例 / 不同缺陷定义**

In [ ]:
import re
def parse_label(label):
    faults = []
    # 提取故障信息：故障N：类型，Bounding Box...
    fault_matches = re.findall(r"故障(\d+)：([^，]+)，Bounding Box([^；]+)", label)
    for match in fault_matches:
        fault_num, fault_type, bbox_desc = match
        # 提取红外和可见光坐标
        ir_coords = re.search(r"在红外图像的坐标为\(([^)]+)\), \(([^)]+)\)", bbox_desc)
        vis_coords = re.search(r"在可见光图像的坐标为\(([^)]+)\), \(([^)]+)\)", bbox_desc)

        ir_coord_str = f"坐标({ir_coords.group(1)})到({ir_coords.group(2)})" if ir_coords else None
        vis_coord_str = f"坐标({vis_coords.group(1)})到({vis_coords.group(2)})" if vis_coords else None

        # 处理“只在红外图像出现”的情况
        if "只在红外图像出现" in bbox_desc:
            ir_coords = re.search(r"只在红外图像出现，坐标为\(([^)]+)\), \(([^)]+)\)", bbox_desc)
            ir_coord_str = f"坐标({ir_coords.group(1)})到({ir_coords.group(2)})" if ir_coords else None
            vis_coord_str = None

        faults.append({
            "type": fault_type.strip(),
            "ir_coords": ir_coord_str,
            "vis_coords": vis_coord_str
        })

    return faults

In [ ]:
# 单个单种缺陷
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)

# 多种不同类型缺陷（草木遮挡+边缘积尘+组串开路）
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
    "故障4：二极管故障，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)

# 多种缺陷（表面破损+多处草木遮挡+边缘积尘+内部缺陷）
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)

label4 = (
    "没有缺陷"
)


In [ ]:
descriptions = {
    "组串短路": {
        "ir_feature": "连续点状热斑",
        "vis_feature": "无明显特征",
        "impact": "会导致光伏面板部分或全部停发，因线路过热形成严重热斑，存在设备烧毁或火灾的潜在风险",
        "peocess priority": 1,
        "reason": "安全风险最高，需要立即处理避免火灾发生",
        "repair_strategy": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，通常无需更换组件。",
        "cause": "常见原因是线路绝缘层破损（如线缆磨损、接头进水）、接线错误（正负极接反）、端子松动短路"
    },
    "组串开路": {
        "ir_feature": "整体热斑",
        "vis_feature": "无明显特征",
        "impact": "会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率",
        "peocess priority": 2,
        "reason": "影响范围最大，需立即排查",
        "repair_strategy": "需要检查汇流箱熔断器是否良好，组件间连接是否正确，组件插头是否松动；修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，通常无需更换组件。",
        "cause": "主要因线路接头松动/脱落、线缆断裂、接线端子氧化接触不良导致"
    },
    "二极管故障": {
        "ir_feature": "条状热斑",
        "vis_feature": "无明显特征",
        "impact": "因热斑效应导致局部发电效率下降30%以上，存在加速组件老化的潜在风险",
        "peocess priority": 3,
        "reason": "可能加速老化，需尽快处理",
        "repair_strategy": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管烧毁且无法单独更换接线盒（或更换成本高），需整体换板。",
        "cause": "部分情况属于线路问题，若二极管外部线路存在虚接、过载，可能导致二极管过流烧毁；但内置二极管自身老化故障不属于线路问题。"
    },
    "内部缺陷": {
        "ir_feature": "点状热斑",
        "vis_feature": "无明显特征",
        "impact": "属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险",
        "peocess priority": 4,
        "reason": "故障持续恶化，无法外部修复",
        "repair_strategy": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        "cause": "如电池片隐裂、断栅、焊带脱落等，通常由于制造缺陷、安装不当或环境因素引起"
    },
    "组件异位": {
        "ir_feature": "大面积区域热斑",
        "vis_feature": "太阳光反射",
        "impact": "因角度偏移导致采光受到影响，可能有10%-30%的发电效率下降，存在结构不稳定的潜在风险",
        "peocess priority": 5,
        "reason": "结构不稳定，有掉落风险，需及时矫正",
        "repair_strategy": "需要评估异位组件损坏情况，并结合IV曲线测试仪组件输出功率测量结果，决定是否更换组件；将原有组件或换新组件安装至正确的位置，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，保证组串连接正常。",
        "cause": "由于安装不稳固、风力或地震等外部力导致角度偏移"
    },
    "表面破损": {
        "ir_feature": "区域热斑",
        "vis_feature": "破损现象",
        "impact": "因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险",
        "peocess priority": 6,
        "reason": "玻璃破碎后会进水、进尘，损坏内部电路，需尽快处理以防止故障扩大。",
        "repair_strategy": "需要及时更换破损组件。",
        "cause": "由于外部冲击、冰雹、动物啃咬等造成玻璃或背板破损"
    },
    "草木遮挡": {
        "ir_feature": "区域热斑",
        "vis_feature": "草木",
        "impact": "会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率",
        "peocess priority": 7,
        "reason": "遮挡面积会随植物生长扩大，快速降低局部发电效率，需定期清理。",
        "repair_strategy": "首先需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用IV曲线测试仪检测组件功率，如低于期望值，则更换组件;如该现象在电站中较普遍，应安排全面除草。",
        "cause": "由于周边植物生长未及时清理导致"
    },
    "固定设施遮挡": {
        "ir_feature": "区域热斑或阴影",
        "vis_feature": "固定设施或阴影",
        "impact": "因固定设施导致光伏板局部面积遮挡，通常引起的发电效率下降小于10%",
        "peocess priority": 8,
        "reason": "遮挡位置固定但持续影响，潜在风险较低，需通过调整设施角度、位置来解决",
        "repair_strategy": "需要排除固定设施遮挡;若无法排除，密切监控组件温度，结合组串电流数据，若组串发电量过低或组件出现过热烧穿等现象，及时更换组件。",
        "cause": "由于安装时未考虑固定设施如支架或围栏的位置导致"
    },
    "表面污迹": {
        "ir_feature": "区域热斑",
        "vis_feature": "污渍",
        "impact": "会轻微影响光伏板透光率，通常引起的发电效率下降小于10%",
        "peocess priority": 9,
        "reason": "潜在风险低，若污迹干燥且无腐蚀性，可按周期清理",
        "repair_strategy": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        "cause": "由于泥土、鸟粪、工业污染物等积累导致"
    },
    "边缘积尘": {
        "ir_feature": "区域热斑",
        "vis_feature": "边缘积尘",
        "impact": "因积尘量少，污染程度较低，通常引起的发电效率下降小于10%",
        "peocess priority": 10,
        "reason": "影响相对较小，可在常规清洁时顺带处理",
        "repair_strategy": "需要清除组件边缘积尘，然后通过红外设备复测组件，通常无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。",
        "cause": "由于灰尘在组件边缘积累，通常因清洁不当或环境尘土多导致"
    },
}

### TypeOne_id3 ——**要是光伏板有隐裂啥的，这两种照片能看出来不？**

In [ ]:
def TypeOne_id3(label):
    # 固定科学说明部分
    science_intro = (
        "光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；"
        "在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。"
    )
    faults = parse_label(label)
    # 检查是否有表面破损
    damage_faults = [f for f in faults if f["type"] == "表面破损"]

    if damage_faults:
        # 有表面破损：合并坐标描述
        ir_coords_list = [df["ir_coords"] for df in damage_faults]
        vis_coords_list = [df["vis_coords"] for df in damage_faults]
        ir_coords_str = "和".join(ir_coords_list)
        vis_coords_str = "和".join(vis_coords_list)
        damage_part = (
            f"对这组图像进行分析，检测到{len(damage_faults)}处表面破损故障，"
            f"在红外图像{'依次' if len(damage_faults) > 1 else ''}呈现在{ir_coords_str}区域，同时对应在可见光图像中{vis_coords_str}的区域，显示出明显的破损特征。"
        )
        answer = f"{science_intro}{damage_part}"
    else:
        # 无表面破损：说明未检测到
        no_damage_desc = (
            "对这组图像进行分析，在红外图像中没有呈现区域光斑，可见光图像中也没有呈现裂纹或碎片等物理损伤迹象，"
            "所以未检测到表面破损。"
        )
        answer = f"{science_intro}{no_damage_desc}"
    return answer

# 测试用例1：单处表面破损
label1 = (
    "总共有2个故障；故障1：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1 输出：")
print(TypeOne_id3(label1))

# 测试用例2：无表面破损
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(325, 282), (338, 291)；"
    "故障3：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("\n测试用例2 输出：")
print(TypeOne_id3(label2))

# 测试用例3：多处表面破损
label3 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：表面破损，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例3 输出：")
print(TypeOne_id3(label3))

测试用例1 输出：
光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。对这组图像进行分析，检测到1处表面破损故障，在红外图像呈现在坐标(365, 148)到(421, 176)区域，同时对应在可见光图像中坐标(1399, 617)到(1598, 725)的区域，显示出明显的破损特征。

测试用例2 输出：
光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。对这组图像进行分析，在红外图像中没有呈现区域光斑，可见光图像中也没有呈现裂纹或碎片等物理损伤迹象，所以未检测到表面破损。

测试用例3 输出：
光伏板的表面破损在红外图像中通常表现为区域光斑，这是因为破损会导致局部热分布不均，形成明显的温度差异区域；在可见光图像中，则可以观察到直接的物理损伤迹象，如裂纹、碎片或表面缺损。对这组图像进行分析，检测到2处表面破损故障，在红外图像依次呈现在坐标(365, 148)到(421, 176)和坐标(200, 300)到(250, 350)区域，同时对应在可见光图像中坐标(1399, 617)到(1598, 725)和坐标(800, 900)到(850, 950)的区域，显示出明显的破损特征。


### TypeOne_D4_Replace
**需要更换哪些东西？**

In [ ]:
def TypeOne_D4_Replace(label):
    # TypeOne_id6
    # 缺陷类型对应的维修措施
    repair_strategies = {
        "草木遮挡": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。",
        "固定设施遮挡": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，无需更换组件。",
        "表面污迹": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        "表面破损": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        "内部缺陷": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        "边缘积尘": "需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。",
        "组件异位": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，无需更换组件。",
        "二极管故障": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        "组串短路": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，无需更换组件。",
        "组串开路": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。"
    }

    faults = parse_label(label)
    print(faults)
    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))  # 去重，获取唯一缺陷类型
    if not fault_types:
        return "对这组图像进行分析，未检测到任何缺陷，无需更换组件或采取维修措施。"

    # 分离高严重性（可能需换板）和低严重性（无需换板）缺陷
    high_severity = ["表面破损", "内部缺陷", "二极管故障"]
    high_severity_faults = [f for f in faults if f["type"] in high_severity]
    low_severity_faults = [f for f in faults if f["type"] not in high_severity]

    # 总述
    answer_parts = []
    answer_parts.append(f"对这组图像进行分析，检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。")

    # 处理高严重性缺陷（可能需换板）
    replace_needed = []
    for fault in high_severity_faults:
        fault_type = fault["type"]
        repair = repair_strategies[fault_type]
        # 优先使用可见光坐标，若无则用红外坐标
        coords = fault["vis_coords"] or fault["ir_coords"] or "指定区域"
        coord_type = "可见光" if fault["vis_coords"] else "热成像"
        replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，{repair}")

    # 处理低严重性缺陷（无需换板），按类型合并
    non_replace_needed = []
    low_severity_types = list(set(f["type"] for f in low_severity_faults))
    if low_severity_types:
        low_severity_desc = []
        for fault_type in low_severity_types:
            # 获取该类型的所有坐标，优先使用可见光坐标
            coords_list = []
            for f in low_severity_faults:
                if f["type"] == fault_type:
                    coord = f["vis_coords"] or f["ir_coords"] or "指定区域"
                    coord_type = "可见光" if f["vis_coords"] else "热成像"
                    coords_list.append(f"{coord_type}{coord}")
            coords_str = "和".join(coords_list)
            repair = repair_strategies[fault_type]
            low_severity_desc.append(f"{fault_type}，出现在在{coords_str}的范围，{repair}")
        non_replace_needed.append(f"{' '.join(low_severity_desc)}")

    # 综合描述
    if replace_needed:
        answer_parts.append("需采取以下维修措施：")
        answer_parts.extend(replace_needed)
        if non_replace_needed:
            answer_parts.append("此外，还需处理以下一般情况下无需更换组件的缺陷：")
            answer_parts.extend(non_replace_needed)
    else:
        answer_parts.append("需采取以下维修措施，通常无需更换组件：")
        answer_parts.extend(non_replace_needed if non_replace_needed else ["无需采取额外维修措施。"])

    return " ".join(answer_parts)

# 测试用例1：单种高严重性缺陷（表面破损）
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1 输出：")
print(TypeOne_D4_Replace(label1))

# 测试用例2：多种不同类型低严重性缺陷（草木遮挡+边缘积尘+组串开路）
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例2 输出：")
print(TypeOne_D4_Replace(label2))

# 测试用例3：多种缺陷（表面破损+多处草木遮挡+边缘积尘+内部缺陷）
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例3 输出：")
print(TypeOne_D4_Replace(label3))

label4 = (
    "没有缺陷"
)
print("测试用例4 输出：")
print(TypeOne_D4_Replace(label4))

测试用例1 输出：
[{'type': '表面破损', 'ir_coords': '坐标(365, 148)到(421, 176)', 'vis_coords': '坐标(1399, 617)到(1598, 725)'}]
对这组图像进行分析，检测到1种缺陷：表面破损。 需采取以下维修措施： 在可见光坐标(1399, 617)到(1598, 725)的范围检测到表面破损，需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。

测试用例2 输出：
[{'type': '草木遮挡', 'ir_coords': '坐标(73, 81)到(110, 108)', 'vis_coords': '坐标(259, 345)到(289, 398)'}, {'type': '边缘积尘', 'ir_coords': '坐标(200, 300)到(250, 350)', 'vis_coords': '坐标(800, 900)到(850, 950)'}, {'type': '组串开路', 'ir_coords': '坐标(557, 155)到(587, 165)', 'vis_coords': None}]
对这组图像进行分析，检测到3种缺陷：草木遮挡、边缘积尘、组串开路。 需采取以下维修措施，通常无需更换组件： 草木遮挡，出现在在可见光坐标(259, 345)到(289, 398)的范围，需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。 边缘积尘，出现在在可见光坐标(800, 900)到(850, 950)的范围，需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。 组串开路，出现在在热成像坐标(557, 155)到(587, 165)的范围，需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。

测试用例3 输出：
[{'type': '表面破损', 'ir_coords': '坐标(365, 148)到(421, 176)', 'vis_coor

### TypeOne_D4_Repair_or_Replace
**这片板子是修理还是直接更换？**

In [ ]:
def TypeOne_D4_Repair_or_Replace(label):
    # 缺陷类型对应的维修措施
    repair_strategies = {
        "草木遮挡": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。",
        "固定设施遮挡": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，无需更换组件。",
        "表面污迹": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        "表面破损": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        "内部缺陷": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        "边缘积尘": "需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。",
        "组件异位": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，无需更换组件。",
        "二极管故障": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        "组串短路": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，无需更换组件。",
        "组串开路": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。"
    }

    faults = parse_label(label)

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    if not fault_types:
        return "对这组图像进行分析，未检测到有任何缺陷，无需修理或更换组件。"

    # 分离高严重性（可能需换板）和低严重性（无需换板）缺陷
    high_severity = ["表面破损", "内部缺陷", "二极管故障"]
    high_severity_faults = [f for f in faults if f["type"] in high_severity]
    low_severity_faults = [f for f in faults if f["type"] not in high_severity]

    # 生成回答
    answer_parts = []
    answer_parts.append(f"对这组图像进行分析，检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。")

    # 处理高严重性缺陷（可能需换板）
    replace_needed = []
    for fault in high_severity_faults:
        fault_type = fault["type"]
        repair = repair_strategies[fault_type]
        coords = fault["vis_coords"] or fault["ir_coords"] or "指定区域"
        coord_type = "可见光" if fault["vis_coords"] else "热成像"
        if fault_type == "表面破损":
            replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，{repair}")
        elif fault_type == "内部缺陷":
            replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，需评估电池片隐裂或断栅程度，若严重则需更换组件。")
        elif fault_type == "二极管故障":
            replace_needed.append(f"在{coord_type}{coords}的范围检测到{fault_type}，需检查接线盒，若二极管不可单独更换则需更换组件，否则可通过更换二极管修复。")

    # 处理低严重性缺陷（无需换板），按类型合并
    non_replace_needed = []
    low_severity_types = list(set(f["type"] for f in low_severity_faults))
    if low_severity_types:
        low_severity_desc = []
        for fault_type in low_severity_types:
            repair = repair_strategies[fault_type]
            coords_list = []
            for f in low_severity_faults:
                if f["type"] == fault_type:
                    coord = f["vis_coords"] or f["ir_coords"] or "指定区域"
                    coord_type = "可见光" if f["vis_coords"] else "热成像"
                    coords_list.append(f"{coord_type}{coord}")
            coords_str = "和".join(coords_list)
            low_severity_desc.append(f"对于{fault_type}，在{coords_str}的范围，{repair}")
        non_replace_needed.append(f"{' '.join(low_severity_desc)}")

    # 综合描述
    if replace_needed:
        answer_parts.append("需评估是否更换组件：")
        answer_parts.extend(replace_needed)
        if non_replace_needed:
            answer_parts.append("其他缺陷通常无需更换组件，只需修理：")
            answer_parts.extend(non_replace_needed)
    else:
        answer_parts.append("一般情况下无需更换组件，只需修理：")
        answer_parts.extend(non_replace_needed if non_replace_needed else ["无需采取额外维修措施。"])

    return " ".join(answer_parts)

# 测试用例1：单种高严重性缺陷（表面破损）
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1 输出：")
print(TypeOne_D4_Repair_or_Replace(label1))

# 测试用例2：多种不同类型低严重性缺陷（草木遮挡+边缘积尘+组串开路）
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例2 输出：")
print(TypeOne_D4_Repair_or_Replace(label2))

# 测试用例3：多种缺陷（表面破损+多处草木遮挡+边缘积尘+内部缺陷）
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("\n测试用例3 输出：")
print(TypeOne_D4_Repair_or_Replace(label3))

label4 = (
    "没有缺陷"
)
print("测试用例4 输出：")
print(TypeOne_D4_Repair_or_Replace(label4))

测试用例1 输出：
对这组图像进行分析，检测到1种缺陷：表面破损。 需评估是否更换组件： 在可见光坐标(1399, 617)到(1598, 725)的范围检测到表面破损，需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。

测试用例2 输出：
对这组图像进行分析，检测到3种缺陷：草木遮挡、边缘积尘、组串开路。 一般情况下无需更换组件，只需修理： 对于草木遮挡，在可见光坐标(259, 345)到(289, 398)的范围，需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。 对于边缘积尘，在可见光坐标(800, 900)到(850, 950)的范围，需要清除组件边缘积尘，然后通过红外设备复测组件，无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。 对于组串开路，在热成像坐标(557, 155)到(587, 165)的范围，需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，无需更换组件。

测试用例3 输出：
对这组图像进行分析，检测到4种缺陷：边缘积尘、草木遮挡、表面破损、内部缺陷。 需评估是否更换组件： 在可见光坐标(1399, 617)到(1598, 725)的范围检测到表面破损，需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。 在热成像坐标(557, 155)到(587, 165)的范围检测到内部缺陷，需评估电池片隐裂或断栅程度，若严重则需更换组件。 其他缺陷通常无需更换组件，只需修理： 对于草木遮挡，在可见光坐标(259, 345)到(289, 398)和可见光坐标(800, 900)到(850, 950)的范围，需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，无需更换组件。 对于边缘积尘，在可见光坐标(900, 1000)到(950, 1050)的范围，需要清除组件边缘积尘

### TypeOne_D3_Power_Failure
**发电系统会瘫痪吗**

In [ ]:
def TypeOne_D3_Power_Failure(label):
    # TypeOne_id7
    # 缺陷类型对应的影响描述
    impact_descriptions = {
        "组串短路": "可能导致部分或全部停发，可能造成发电系统局部或全部瘫痪。",
        "组串开路": "导致整串停发，可能造成发电系统局部或全部瘫痪。",
        "二极管故障": "热斑导致局部效率下降30%以上，可能造成发电系统局部瘫痪。",
        "内部缺陷": "效率下降10%-50%，可能造成发电系统局部瘫痪。",
        "组件异位": "角度偏移导致采光减少，效率下降10%-30%，通常不会导致系统瘫痪。",
        "表面破损": "透光率下降，效率下降10%-30%，通常不会导致系统瘫痪。",
        "草木遮挡": "遮挡区域发电归零并影响周边组件，效率下降10%-30%，通常不会导致系统瘫痪。",
        "固定设施遮挡": "局部小面积遮挡，效率下降小于10%，通常不会导致系统瘫痪。",
        "表面污迹": "透光率轻微下降，效率下降小于10%，通常不会导致系统瘫痪。",
        "边缘积尘": "边缘少量积尘，效率下降小于10%，通常不会导致系统瘫痪。"
    }

    faults = parse_label(label)

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    if not fault_types:
        return "对这组图像进行分析，未检测到任何缺陷，发电系统正常运行，无瘫痪风险。"

    # 分组：严重影响（可能瘫痪）、中度影响、轻度影响
    severe_impact = ["组串短路", "组串开路", "二极管故障", "内部缺陷"]
    moderate_impact = ["组件异位", "表面破损", "草木遮挡"]
    low_impact = ["固定设施遮挡", "表面污迹", "边缘积尘"]

    severe_faults = [f for f in faults if f["type"] in severe_impact]
    moderate_faults = [f for f in faults if f["type"] in moderate_impact]
    low_faults = [f for f in faults if f["type"] in low_impact]

    # 生成回答
    answer_parts = []
    answer_parts.append(f"对这组图像进行分析，检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。")

    # 处理严重影响缺陷（可能导致瘫痪）
    severe_desc = []
    for fault in severe_faults:
        fault_type = fault["type"]
        coords = fault["vis_coords"] or fault["ir_coords"] or "指定区域"
        coord_type = "可见光" if fault["vis_coords"] else "热成像"
        severe_desc.append(f"在{coord_type}{coords}的范围检测到{fault_type}，{impact_descriptions[fault_type]}")

    # 处理中度/轻度影响缺陷（不会瘫痪）
    moderate_low_desc = []
    moderate_low_types = list(set(f["type"] for f in moderate_faults + low_faults))
    for fault_type in moderate_low_types:
        coords_list = []
        for f in moderate_faults + low_faults:
            if f["type"] == fault_type:
                coord = f["vis_coords"] or f["ir_coords"] or "指定区域"
                coord_type = "可见光" if f["vis_coords"] else "热成像"
                coords_list.append(f"{coord_type}{coord}")
        if coords_list:
            coords_str = "和".join(coords_list)
            moderate_low_desc.append(f"对于{fault_type}，在{coords_str}的范围，{impact_descriptions[fault_type]}")

    # 综合描述
    if severe_desc:
        answer_parts.append("以下缺陷可能导致发电系统瘫痪：")
        answer_parts.extend(severe_desc)
        if moderate_low_desc:
            answer_parts.append("其他缺陷一般不会导致系统瘫痪，但会降低发电效率：")
            answer_parts.extend(moderate_low_desc)
    else:
        answer_parts.append("这种缺陷一般不会导致发电系统瘫痪，但会降低发电效率：")
        answer_parts.extend(moderate_low_desc if moderate_low_desc else ["未检测到影响发电的重大缺陷。"])

    return "".join(answer_parts)

print("测试用例1 输出：")
print(TypeOne_D3_Power_Failure(label1))

print("\n测试用例2 输出：")
print(TypeOne_D3_Power_Failure(label2))

print("\n测试用例3 输出：")
print(TypeOne_D3_Power_Failure(label3))

print("测试用例4 输出：")
print(TypeOne_D3_Power_Failure(label4))

测试用例1 输出：
对这组图像进行分析，检测到1种缺陷：表面破损。这种缺陷一般不会导致发电系统瘫痪，但会降低发电效率：对于表面破损，在可见光坐标(1399, 617)到(1598, 725)的范围，透光率下降，效率下降10%-30%，通常不会导致系统瘫痪。

测试用例2 输出：
对这组图像进行分析，检测到3种缺陷：草木遮挡、边缘积尘、组串开路。以下缺陷可能导致发电系统瘫痪：在热成像坐标(557, 155)到(587, 165)的范围检测到组串开路，导致整串停发，可能造成发电系统局部或全部瘫痪。其他缺陷一般不会导致系统瘫痪，但会降低发电效率：对于草木遮挡，在可见光坐标(259, 345)到(289, 398)的范围，遮挡区域发电归零并影响周边组件，效率下降10%-30%，通常不会导致系统瘫痪。对于边缘积尘，在可见光坐标(800, 900)到(850, 950)的范围，边缘少量积尘，效率下降小于10%，通常不会导致系统瘫痪。

测试用例3 输出：
对这组图像进行分析，检测到4种缺陷：边缘积尘、草木遮挡、表面破损、内部缺陷。以下缺陷可能导致发电系统瘫痪：在热成像坐标(557, 155)到(587, 165)的范围检测到内部缺陷，效率下降10%-50%，可能造成发电系统局部瘫痪。其他缺陷一般不会导致系统瘫痪，但会降低发电效率：对于边缘积尘，在可见光坐标(900, 1000)到(950, 1050)的范围，边缘少量积尘，效率下降小于10%，通常不会导致系统瘫痪。对于草木遮挡，在可见光坐标(259, 345)到(289, 398)和可见光坐标(800, 900)到(850, 950)的范围，遮挡区域发电归零并影响周边组件，效率下降10%-30%，通常不会导致系统瘫痪。对于表面破损，在可见光坐标(1399, 617)到(1598, 725)的范围，透光率下降，效率下降10%-30%，通常不会导致系统瘫痪。
测试用例4 输出：
对这组图像进行分析，未检测到任何缺陷，发电系统正常运行，无瘫痪风险。


### TypeOne_D6_Object_or_Panal
**红外图上有阴影是否能辨别出是遮挡问题还是板子问题？**

In [ ]:
import re
# 红外图上有阴影是否能辨别出是遮挡问题还是板子问题？ TypeOne_id12
def TypeOne_D6_Object_or_Panal(label):
    # 固定科学说明部分，聚焦于遮挡但一般化以覆盖板子问题
    science_intro = (
        "这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。"
        "如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；"
        "在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。"
    )

    # 缺陷特征描述字典
    feature_descriptions = {
        "草木遮挡": {"ir": "区域光斑", "vis": "草木", "category": "遮挡"},
        "固定设施遮挡": {"ir": "区域光斑或阴影", "vis": "固定设施或阴影", "category": "遮挡"},
        "表面污迹": {"ir": "区域光斑", "vis": "污渍", "category": "板子"},
        "表面破损": {"ir": "区域光斑", "vis": "破损", "category": "板子"},
        "内部缺陷": {"ir": "点状光斑", "vis": "无缺陷", "category": "板子"},
        "边缘积尘": {"ir": "区域光斑", "vis": "组件有边缘积尘", "category": "板子"},
        "组件异位": {"ir": "大面积区域光斑", "vis": "太阳光反射", "category": "板子"},
        "二极管故障": {"ir": "条状光斑", "vis": "无缺陷", "category": "板子"},
        "组串短路": {"ir": "光伏组件连续点状光斑", "vis": "无缺陷", "category": "板子"},
        "组串开路": {"ir": "光伏组件整体出现光斑", "vis": "无缺陷", "category": "板子"},
    }

    faults = parse_label(label)

    if not faults:
        analysis_part = "对这组图像进行分析，没有检测到缺陷特征，没有遮挡问题也没有板子问题。"
    else:
        analysis_part = "对这组图像进行分析，"
        descs = []
        for f in faults:
            fault_type = f["type"]
            if fault_type in feature_descriptions:
                ir_coords = f["ir_coords"]
                vis_coords = f["vis_coords"]
                ir_feature = feature_descriptions[fault_type]["ir"]
                vis_feature = feature_descriptions[fault_type]["vis"]
                category = feature_descriptions[fault_type]["category"]
                vis_desc = f"可见光图像{vis_coords}的范围识别到{vis_feature}" if vis_coords else f"可见光图像{vis_feature}"
                conclusion = "因此是遮挡问题而非板子问题" if category == "遮挡" else "因此是板子问题而非遮挡问题"
                desc = f"在红外图像{ir_coords}的区域，有{ir_feature}，而在{vis_desc}，可以辨别出是{fault_type}现象，{conclusion}"
                descs.append(desc)
        if descs:
            analysis_part += "；".join(descs) + "。"
        # else:
        #     analysis_part += "未检测到任何缺陷，因此无法明确辨别是遮挡问题还是板子问题。"

    answer = f"{science_intro} {analysis_part}"

    return answer

# 测试示例1：有遮挡
label1 = "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(325, 282), (338, 291)；故障3：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
print("示例1输出：")
print(TypeOne_D6_Object_or_Panal(label1))

# 测试示例2：有板子问题
label2 = "总共有2个故障；故障1：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；故障2：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
print("\n示例2输出：")
print(TypeOne_D6_Object_or_Panal(label2))

# 测试示例3：无相关缺陷或混合
label3 = "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
print("\n示例3输出：")
print(TypeOne_D6_Object_or_Panal(label3))

print("测试用例4 输出：")
print(TypeOne_D6_Object_or_Panal(label4))

示例1输出：
这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。 对这组图像进行分析，在红外图像坐标(73, 81)到(110, 108)的区域，有区域光斑，而在可见光图像坐标(259, 345)到(289, 398)的范围识别到草木，可以辨别出是草木遮挡现象，因此是遮挡问题而非板子问题；在红外图像坐标(325, 282)到(338, 291)的区域，有点状光斑，而在可见光图像无缺陷，可以辨别出是内部缺陷现象，因此是板子问题而非遮挡问题；在红外图像坐标(532, 24)到(621, 38)的区域，有点状光斑，而在可见光图像无缺陷，可以辨别出是内部缺陷现象，因此是板子问题而非遮挡问题。

示例2输出：
这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。 对这组图像进行分析，在红外图像坐标(557, 155)到(587, 165)的区域，有点状光斑，而在可见光图像无缺陷，可以辨别出是内部缺陷现象，因此是板子问题而非遮挡问题；在红外图像坐标(365, 148)到(421, 176)的区域，有区域光斑，而在可见光图像坐标(1399, 617)到(1598, 725)的范围识别到破损，可以辨别出是表面破损现象，因此是板子问题而非遮挡问题。

示例3输出：
这个问题需要通过红外图像的特征和可见光图像的对应迹象来判断。如果发生遮挡现象，红外图像上通常表现为区域光斑，这是因为遮挡会导致局部热分布不均，形成温度差异区域；在可见光图像中，则可以观察到草木或固定设施等外部物体迹象，从而确认是遮挡问题还是板子内部问题。 对这组图像进行分析，在红外图像坐标(73, 81)到(110, 108)的区域，有区域光斑，而在可见光图像坐标(259, 345)到(289, 398)的范围识别到组件有边缘积尘，可以辨别出是边缘积尘现象，因此是板子问题而非遮挡问题。
测试用例4 输出：
这个问题需要通过红

### TypeOne_D7_Hotpot_Imapact
**这张照片中的热斑会造成上面影响?**

In [ ]:
def TypeOne_id13(label):
    """
    这张照片中的热斑会造成上面影响? TypeOne_id13
    """
    # 固定科学说明部分
    science_intro = (
        "红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。"
    )

    # 缺陷特征和影响描述字典
    descriptions = {
        "草木遮挡": {
            "ir_feature": "区域光斑",
            "vis_feature": "草木",
            "impact": "会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率"
        },
        "固定设施遮挡": {
            "ir_feature": "区域光斑或阴影",
            "vis_feature": "固定设施或阴影",
            "impact": "如果是局部小面积遮挡，效率下降小于10%，通常不会直接导致系统瘫痪"
        },
        "表面污迹": {
            "ir_feature": "区域光斑",
            "vis_feature": "污渍",
            "impact": "会引起透光率轻微下降，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪"
        },
        "表面破损": {
            "ir_feature": "区域光斑",
            "vis_feature": "破损",
            "impact": "会导致透光率下降，致使10%-30%的发电效率下降，通常不会直接导致系统瘫痪"
        },
        "内部缺陷": {
            "ir_feature": "点状光斑",
            "vis_feature": "无明显特征",
            "impact": "影响较大，会导致发电效率下降10%-50%，可能造成发电系统局部瘫痪"
        },
        "边缘积尘": {
            "ir_feature": "区域光斑",
            "vis_feature": "边缘积尘",
            "impact": "边缘少量积尘，对整体透光影响较小，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪"
        },
        "组件异位": {
            "ir_feature": "大面积区域光斑",
            "vis_feature": "太阳光反射",
            "impact": "角度偏移导致采光减少，效率下降10%-30%，通常不会导致系统瘫痪"
        },
        "二极管故障": {
            "ir_feature": "条状光斑",
            "vis_feature": "无明显特征",
            "impact": "热斑会导致局部效率下降30%以上，可能造成发电系统局部瘫痪"
        },
        "组串短路": {
            "ir_feature": "光伏组件连续点状光斑",
            "vis_feature": "无明显特征",
            "impact": "可能导致部分或全部停发，可能造成发电系统局部或全部瘫痪"
        },
        "组串开路": {
            "ir_feature": "光伏组件整体出现光斑",
            "vis_feature": "无明显特征",
            "impact": "导致整串停发，可能造成发电系统局部或全部瘫痪"
        },
    }

    # 解析label
    faults = parse_label(label)

    # 生成分析部分
    analysis_part = "对这组图像进行分析，"

    if not faults:
        analysis_part += "未检测到热斑相关特征，无明显影响。"
    else:
        descs = []
        for f in faults:
            fault_type = f["type"]
            if fault_type in descriptions:
                ir_coords = f["ir_coords"]
                vis_coords = f["vis_coords"]
                ir_feature = descriptions[fault_type]["ir_feature"]
                vis_feature = descriptions[fault_type]["vis_feature"]
                impact = descriptions[fault_type]["impact"]
                vis_desc = f"而在可见光图像对应区域有{vis_feature}" if vis_coords else f"而在可见光图像对应区域{vis_feature}"
                desc = f"在红外图像{ir_coords}的范围，存在{ir_feature}，{vis_desc}，对应{fault_type}的特征，{impact}"
                descs.append(desc)
        if descs:
            analysis_part += "；".join(descs) + "。"
        else:
            analysis_part += "未检测到热斑相关特征，无明显影响。"

    answer = f"{science_intro} {analysis_part}"

    return answer

# 测试用例1：严重影响（内部缺陷+组串开路）
label1 = (
    "总共有2个故障；故障1：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：组串开路，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_id13(label1))

# 测试用例2：中度+轻度影响（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_id13(label2))

# 测试用例3：无热斑相关缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_id13(label3))

print("测试用例4 输出：")
print(TypeOne_id13(label4))

测试用例1 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行分析，在红外图像坐标(557, 155)到(587, 165)的范围，存在点状光斑，而在可见光图像对应区域无明显特征，对应内部缺陷的特征，影响较大，会导致发电效率下降10%-50%，可能造成发电系统局部瘫痪；在红外图像坐标(532, 24)到(621, 38)的范围，存在光伏组件整体出现光斑，而在可见光图像对应区域无明显特征，对应组串开路的特征，导致整串停发，可能造成发电系统局部或全部瘫痪。

测试用例2 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行分析，在红外图像坐标(365, 148)到(421, 176)的范围，存在区域光斑，而在可见光图像对应区域有破损，对应表面破损的特征，会导致透光率下降，致使10%-30%的发电效率下降，通常不会直接导致系统瘫痪；在红外图像坐标(73, 81)到(110, 108)的范围，存在区域光斑，而在可见光图像对应区域有草木，对应草木遮挡的特征，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率；在红外图像坐标(300, 400)到(350, 450)的范围，存在区域光斑，而在可见光图像对应区域有边缘积尘，对应边缘积尘的特征，边缘少量积尘，对整体透光影响较小，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪。

测试用例3 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行分析，在红外图像坐标(73, 81)到(110, 108)的范围，存在区域光斑，而在可见光图像对应区域有边缘积尘，对应边缘积尘的特征，边缘少量积尘，对整体透光影响较小，造成的发电效率下降一般小于10%，通常不会直接导致系统瘫痪。
测试用例4 输出：
红外图像中的热斑是高温异常区域，通常由缺陷引起，不同缺陷会表现不同的热斑特征，可能导致发电效率下降、安全隐患或系统瘫痪等不同程度的影响。 对这组图像进行

### TypeOne_D1_Defect_Priority
**从图上看，最要紧的问题是哪个呀？**

In [ ]:
def TypeOne_D1_Defect_Priority(label):
    """
    “从图上看，最要紧的问题是哪个呀？” TypeOne_id66
    """
    # 缺陷影响和优先级描述字典
    descriptions = {
        "组串短路": {
            "impact": "可能导致线路过热、设备烧毁甚至火灾，存在严重安全隐患",
            "priority": 1,
            "reason": "安全风险最高，可能引发火灾"
        },
        "组串开路": {
            "impact": "导致整串停止发电，显著降低系统输出",
            "priority": 2,
            "reason": "影响范围最大，需立即排查"
        },
        "二极管故障": {
            "impact": "引发热斑，导致局部效率下降30%以上，加速组件老化",
            "priority": 3,
            "reason": "热斑加速老化，需尽快处理"
        },
        "内部缺陷": {
            "impact": "效率下降10%-50%，可能导致局部瘫痪，故障持续恶化",
            "priority": 4,
            "reason": "隐性故障持续恶化，无法外部修复"
        },
        "组件异位": {
            "impact": "角度偏移减少采光，效率下降10%-30%，可能存在结构不稳定风险",
            "priority": 5,
            "reason": "结构不稳定需及时矫正"
        },
        "表面破损": {
            "impact": "透光率下降，效率下降10%-30%，可能因进水损坏内部电路",
            "priority": 6,
            "reason": "可能导致故障扩大，需尽快处理"
        },
        "草木遮挡": {
            "impact": "遮挡区域发电归零，效率下降10%-30%，影响随植物生长扩大",
            "priority": 7,
            "reason": "遮挡面积快速扩大，需定期清理"
        },
        "固定设施遮挡": {
            "impact": "局部小面积遮挡，效率下降小于10%，影响持续但固定",
            "priority": 8,
            "reason": "影响较小，可调整设施位置"
        },
        "表面污迹": {
            "impact": "透光率轻微下降，效率下降小于10%，无腐蚀性可按周期清理",
            "priority": 9,
            "reason": "影响轻微，可周期清理"
        },
        "边缘积尘": {
            "impact": "边缘少量积尘，效率下降小于10%，影响最小",
            "priority": 10,
            "reason": "影响最小，可常规清理"
        },
    }

    # 解析label
    faults = parse_label(label)

    # 生成回答
    if not faults:
        return "未检测到任何缺陷，无需优先处理任何问题。"

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    answer_parts = [f"检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。"]

    # 确定最要紧问题
    priority_faults = [(f["type"], descriptions.get(f["type"], {"priority": 99})["priority"], f["ir_coords"]) for f in faults]
    most_urgent = min(priority_faults, key=lambda x: x[1])
    urgent_type, urgent_priority, urgent_coords = most_urgent

    # 最要紧问题描述
    if urgent_type in descriptions:
        urgent_impact = descriptions[urgent_type]["impact"]
        urgent_reason = descriptions[urgent_type]["reason"]
        answer_parts.append(
            f"最要紧的问题是{urgent_type}，在红外图像{urgent_coords}的范围，"
            f"{urgent_impact}，因为{urgent_reason}。"
        )
    else:
        answer_parts.append("未检测到相关缺陷特征，无需优先处理任何问题。")
        return " ".join(answer_parts)

    # 其他缺陷描述，按优先级排序
    other_faults = [(f["type"], descriptions.get(f["type"], {"priority": 99})["priority"], f["ir_coords"])
                    for f in faults if f["type"] != urgent_type]
    if other_faults:
        answer_parts.append("其他缺陷的影响如下：")
        # 按优先级排序
        other_faults.sort(key=lambda x: x[1])
        prev_priority = None
        for fault_type, priority, coords in other_faults:
            if fault_type in descriptions:
                impact = descriptions[fault_type]["impact"]
                if prev_priority is None or priority <= 4:
                    desc = f"{fault_type}在红外图像{coords}的范围，{impact}。"
                elif priority <= 7:
                    desc = f"相较之下，{fault_type}在红外图像{coords}的范围，{impact}。"
                else:
                    desc = f"此外，{fault_type}在红外图像{coords}的范围，{impact}。"
                answer_parts.append(desc)
                prev_priority = priority

    return " ".join(answer_parts)

# 测试用例1：高优先级缺陷（组串短路+内部缺陷）
label1 = (
    "总共有2个故障；故障1：组串短路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_D1_Defect_Priority(label1))

# 测试用例2：中度+低优先级缺陷（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_D1_Defect_Priority(label2))

# 测试用例3：单一低优先级缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_D1_Defect_Priority(label3))

测试用例1 输出：
检测到2种缺陷：组串短路、内部缺陷。 最要紧的问题是组串短路，在红外图像坐标(557, 155)到(587, 165)的范围，可能导致线路过热、设备烧毁甚至火灾，存在严重安全隐患，因为安全风险最高，可能引发火灾。 其他缺陷的影响如下： 内部缺陷在红外图像坐标(532, 24)到(621, 38)的范围，效率下降10%-50%，可能导致局部瘫痪，故障持续恶化。

测试用例2 输出：
检测到3种缺陷：边缘积尘、草木遮挡、表面破损。 最要紧的问题是表面破损，在红外图像坐标(365, 148)到(421, 176)的范围，透光率下降，效率下降10%-30%，可能因进水损坏内部电路，因为可能导致故障扩大，需尽快处理。 其他缺陷的影响如下： 草木遮挡在红外图像坐标(73, 81)到(110, 108)的范围，遮挡区域发电归零，效率下降10%-30%，影响随植物生长扩大。 此外，边缘积尘在红外图像坐标(300, 400)到(350, 450)的范围，边缘少量积尘，效率下降小于10%，影响最小。

测试用例3 输出：
检测到1种缺陷：边缘积尘。 最要紧的问题是边缘积尘，在红外图像坐标(73, 81)到(110, 108)的范围，边缘少量积尘，效率下降小于10%，影响最小，因为影响最小，可常规清理。


### TypeOne_D4_Repair_Tools
**修复这块光伏板的热斑问题需要带什么工具？**

In [ ]:
def TypeOne_D4_Repair_Tools(label):
    """
    “修复这块光伏板的热斑问题需要带什么工具？”
    """
    # 缺陷优先级和工具提取
    defect_info = {
        "组串短路": {
            "tools": ["汇流箱熔断器", "绝缘电阻测试仪", "红外热像仪"],
            "strategy": "需要先使用汇流箱熔断器断开短路组串回路，排查并修复短路点，然后用绝缘电阻测试仪验证无短路，最后用红外热像仪确认连续点状热斑消失，一般无需更换组件",
            "priority": 1
        },
        "组串开路": {
            "tools": ["电压表", "红外热像仪"],
            "strategy": "需要先检查汇流箱熔断器、组件间连接和插头，修复开路点后用电压表测试组串连续性，最后用红外热像仪确认整体热斑消除，一般无需更换组件",
            "priority": 2
        },
        "二极管故障": {
            "tools": ["多用表", "红外热像仪", "备用二极管"],
            "strategy": "需要先检查接线盒，用多用表测试二极管导通，若可更换则使用备用二极管修复，之后用红外热像仪确认条状热斑消除；若不可更换，则需考虑更换光伏组件",
            "priority": 3
        },
        "内部缺陷": {
            "tools": ["EL电致发光测试仪", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "需要先用EL电致发光测试仪、红外热像仪或I-V曲线测试仪确认缺陷范围和功率衰减，若严重则需考虑更换光伏组件",
            "priority": 4
        },
        "组件异位": {
            "tools": ["螺丝刀", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "先用螺丝刀重新固定或对齐异位组件，确保支架稳固，然后用红外热像仪检查大面积热斑消失，最后用I-V曲线测试仪验证电压电流平衡，一般无需更换组件",
            "priority": 5
        },
        "表面破损": {
            "tools": ["专用修补胶", "密封剂", "红外热像仪", "电绝缘测试仪"],
            "strategy": "需要先评估破损程度，若轻微划痕则用专用修补胶或密封剂修复玻璃或背板，之后用红外热像仪和电绝缘测试仪检查无热斑或漏电风险；若严重则需考虑更换光伏组件",
            "priority": 6
        },
        "草木遮挡": {
            "tools": ["镰刀", "剪刀", "I-V曲线测试仪", "红外热像仪"],
            "strategy": "需要先用镰刀或剪刀清除杂草和树枝，然后用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，一般无需更换组件",
            "priority": 7
        },
        "固定设施遮挡": {
            "tools": ["螺丝刀", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "需要先用螺丝刀移除或调整固定设施如支架或围栏，若无法移除则重新布局组件，然后用红外热像仪验证阴影消除，最后用I-V曲线测试仪确认发电效率恢复，一般无需更换组件",
            "priority": 8
        },
        "表面污迹": {
            "tools": ["水", "中性清洁剂", "红外热像仪", "I-V曲线测试仪"],
            "strategy": "需要先用水或中性清洁剂去除污渍，然后用红外热像仪复测组件，若热斑仍存在则用I-V曲线测试仪检测功率，若功率低于期望值则需考虑更换组件",
            "priority": 9
        },
        "边缘积尘": {
            "tools": ["清洁工具", "红外热像仪"],
            "strategy": "需要先用清洁工具清除边缘积尘，然后用红外热像仪复测组件，一般无需更换组件",
            "priority": 10
        },
    }

    # 解析label
    faults = parse_label(label)
    print(faults)

    # 生成回答
    if not faults:
        return "对这组图像进行分析，未检测到任何热斑相关缺陷，无需携带工具进行修复。"

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    answer_parts = [f"对这组图像进行分析，共检测到{len(fault_types)}种缺陷：{'、'.join(fault_types)}。修复热斑问题需携带以下工具："]

    # 按优先级排序故障
    priority_faults = [(f["type"], defect_info.get(f["type"], {"priority": 99})["priority"], f["ir_coords"])
                      for f in faults]
    priority_faults.sort(key=lambda x: x[1])

    # 描述每个缺陷的工具需求
    tool_descs = []
    all_tools = set()
    for fault_type, priority, ir_coords in priority_faults:
        if fault_type in defect_info:
            tools = defect_info[fault_type]["tools"]
            strategy = defect_info[fault_type]["strategy"]
            all_tools.update(tools)
            if priority <= 4:  # 高优先级缺陷，详细描述
                desc = f"对于出现在红外图像{ir_coords}范围的{fault_type}，{strategy}，需携带{'、'.join(tools)}。"
            else:  # 中低优先级，简述
                desc = f"对于出现在红外图像{ir_coords}范围的{fault_type}，{strategy}，需携带{'、'.join(tools)}。"
            tool_descs.append(desc)

    # 综合工具列表
    answer_parts.extend(tool_descs)
    answer_parts.append(f"总结所需工具有{'、'.join(sorted(all_tools))}。")


    return "".join(answer_parts)

# 测试用例1：高优先级缺陷（组串短路+内部缺陷）
label1 = (
    "总共有2个故障；故障1：组串短路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_D4_Repair_Tools(label1))

# 测试用例2：中度+低优先级缺陷（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_D4_Repair_Tools(label2))

# 测试用例3：单一低优先级缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_D4_Repair_Tools(label3))

测试用例1 输出：
[{'type': '组串短路', 'ir_coords': '坐标(557, 155)到(587, 165)', 'vis_coords': None}, {'type': '内部缺陷', 'ir_coords': '坐标(532, 24)到(621, 38)', 'vis_coords': None}]
对这组图像进行分析，共检测到2种缺陷：组串短路、内部缺陷。修复热斑问题需携带以下工具：对于出现在红外图像坐标(557, 155)到(587, 165)范围的组串短路，需要先使用汇流箱熔断器断开短路组串回路，排查并修复短路点，然后用绝缘电阻测试仪验证无短路，最后用红外热像仪确认连续点状热斑消失，一般无需更换组件，需携带汇流箱熔断器、绝缘电阻测试仪、红外热像仪。对于出现在红外图像坐标(532, 24)到(621, 38)范围的内部缺陷，需要先用EL电致发光测试仪、红外热像仪或I-V曲线测试仪确认缺陷范围和功率衰减，若严重则需考虑更换光伏组件，需携带EL电致发光测试仪、红外热像仪、I-V曲线测试仪。总结所需工具有EL电致发光测试仪、I-V曲线测试仪、汇流箱熔断器、红外热像仪、绝缘电阻测试仪。

测试用例2 输出：
[{'type': '表面破损', 'ir_coords': '坐标(365, 148)到(421, 176)', 'vis_coords': '坐标(1399, 617)到(1598, 725)'}, {'type': '草木遮挡', 'ir_coords': '坐标(73, 81)到(110, 108)', 'vis_coords': '坐标(259, 345)到(289, 398)'}, {'type': '边缘积尘', 'ir_coords': '坐标(300, 400)到(350, 450)', 'vis_coords': '坐标(900, 1000)到(950, 1050)'}]
对这组图像进行分析，共检测到3种缺陷：边缘积尘、草木遮挡、表面破损。修复热斑问题需携带以下工具：对于出现在红外图像坐标(365, 148)到(421, 176)范围的表面破损，需要先评估破损程度，若轻微划痕则用专用修补胶或密封剂修复玻璃或背板，之后用红外热像仪和电绝缘测试仪检查无热斑或漏电风险；若严重则需考虑更换光伏组件，需携带专用修补胶、密封剂、红

### TypeOne_D4_Process_Priority
**哪些问题是需要最优先处理的？**

In [ ]:
def TypeOne_D4_Process_Priority(label):
    # TypeOne_id22
    # 缺陷特征、缺陷影响、处理优先级、处理原因、维修策略描述
    descriptions = {
        "组串短路": {
            "ir_feature": "连续点状热斑",
            "vis_feature": "无明显特征",
            "impact": "会导致光伏面板部分或全部停发，因线路过热形成严重热斑，存在设备烧毁或火灾的潜在风险",
            "peocess priority": 1,
            "reason": "安全风险最高，需要立即处理避免火灾发生",
            "repair_strategy": "需要通过断开汇流箱熔断器断开短路组串回路，排查并修复短路故障点，然后使用绝缘电阻测试仪验证无短路并通过红外热像仪确认连续点状热斑消失，通常无需更换组件。",
        },
        "组串开路": {
            "ir_feature": "整体热斑",
            "vis_feature": "无明显特征",
            "impact": "会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率",
            "peocess priority": 2,
            "reason": "影响范围最大，需立即排查",
            "repair_strategy": "需要检查汇流箱熔断器、组件间连接和插头，修复开路点后使用电压表测试组串连续性并通过红外热像仪确认整体热斑消除，通常无需更换组件。"
        },
        "二极管故障": {
            "ir_feature": "条状热斑",
            "vis_feature": "无明显特征",
            "impact": "因热斑效应导致局部发电效率下降30%以上，存在加速组件老化的潜在风险",
            "peocess priority": 3,
            "reason": "可能加速老化，需尽快处理",
            "repair_strategy": "需要检查接线盒，若二极管可单独更换则进行更换，然后使用多用表测试二极管导通并通过红外热像仪确认条状热斑消除；若二极管不可单独更换或更换成本过高，则需更换组件。",
        },
        "内部缺陷": {
            "ir_feature": "点状热斑",
            "vis_feature": "无明显特征",
            "impact": "属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险",
            "peocess priority": 4,
            "reason": "故障持续恶化，无法外部修复",
            "repair_strategy": "需要使用EL电致发光测试、红外热像仪或I-V曲线分析确认缺陷范围和功率衰减程度，若为电池片隐裂、断栅、焊带脱落等严重问题，则需更换组件。",
        },
        "组件异位": {
            "ir_feature": "大面积区域热斑",
            "vis_feature": "太阳光反射",
            "impact": "因角度偏移导致采光受到影响，可能有10%-30%的发电效率下降，存在结构不稳定的潜在风险",
            "peocess priority": 5,
            "reason": "结构不稳定，有掉落风险，需及时矫正",
            "repair_strategy": "需要重新固定或对齐异位组件，确保安装支架稳固，然后使用红外热像仪检查大面积热斑消失并测试组串电压电流平衡，通常无需更换组件。",
        },
        "表面破损": {
            "ir_feature": "区域热斑",
            "vis_feature": "破损现象",
            "impact": "因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险",
            "peocess priority": 6,
            "reason": "玻璃破碎后会进水、进尘，损坏内部电路，需尽快处理以防止故障扩大。",
            "repair_strategy": "需要评估破损程度，若为轻微划痕则使用专用修补胶或密封剂修复玻璃或背板，然后使用红外热像仪和电绝缘测试仪检查修复后无热斑或漏电风险；若破损严重（如大面积玻璃破碎或背板开裂），则需更换组件。",
        },
        "草木遮挡": {
            "ir_feature": "区域热斑",
            "vis_feature": "草木",
            "impact": "会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率",
            "peocess priority": 7,
            "reason": "遮挡面积会随植物生长扩大，快速降低局部发电效率，需定期清理。",
            "repair_strategy": "需要使用镰刀或剪刀清除组件表面的杂草和树枝，然后使用I-V曲线测试仪或红外热像仪检查组件性能和温度分布，确保无剩余热斑，通常无需更换组件。",
        },
        "固定设施遮挡": {
            "ir_feature": "区域热斑或阴影",
            "vis_feature": "固定设施或阴影",
            "impact": "因固定设施导致光伏板局部面积遮挡，通常引起的发电效率下降小于10%",
            "peocess priority": 8,
            "reason": "遮挡位置固定但持续影响，潜在风险较低，需通过调整设施角度、位置来解决",
            "repair_strategy": "需要评估并移除或调整固定设施如支架或围栏，若无法移除则重新布局组件位置，然后使用红外热像仪验证阴影消除并进行I-V曲线测试确认发电效率恢复，通常无需更换组件。",
        },
        "表面污迹": {
            "ir_feature": "区域热斑",
            "vis_feature": "污渍",
            "impact": "会轻微影响光伏板透光率，通常引起的发电效率下降小于10%",
            "peocess priority": 9,
            "reason": "潜在风险低，若污迹干燥且无腐蚀性，可按周期清理",
            "repair_strategy": "需要使用水或中性清洁剂去除组件表面污渍，然后在辐照度较好的环境下，用红外设备复测组件，如热斑依然存在，使用I-V曲线测试仪检测组件功率，若功率低于期望值，则需更换组件。",
        },
        "边缘积尘": {
            "ir_feature": "区域热斑",
            "vis_feature": "边缘积尘",
            "impact": "因积尘量少，污染程度较低，通常引起的发电效率下降小于10%",
            "peocess priority": 10,
            "reason": "影响相对较小，可在常规清洁时顺带处理",
            "repair_strategy": "需要清除组件边缘积尘，然后通过红外设备复测组件，通常无需更换组件。如现象普遍，应对全站组件边缘进行重点清洗。"
        },
    }

    # 解析label
    faults = parse_label(label)

    # 生成回答
    if not faults:
        return "未检测到任何异常情况，无需优先处理任何问题。"

    # 检测到的缺陷类型
    fault_types = list(set(f["type"] for f in faults))
    answer_parts = [f"对这组图像进行分析，共检测到{len(fault_types)}种异常情况：{'、'.join(fault_types)}。"]

    # 按优先级排序故障
    priority_faults = [(f["type"], descriptions.get(f["type"], {"peocess priority": 99})["peocess priority"],
                       f["ir_coords"], f["vis_coords"]) for f in faults]
    priority_faults.sort(key=lambda x: x[1])

    # 最优先问题
    most_urgent = priority_faults[0]
    urgent_type, urgent_priority, urgent_ir_coords, urgent_vis_coords = most_urgent
    if urgent_type in descriptions:
        ir_feature = descriptions[urgent_type]["ir_feature"]
        vis_feature = descriptions[urgent_type]["vis_feature"]
        impact = descriptions[urgent_type]["impact"]
        reason = descriptions[urgent_type]["reason"]
        vis_desc = f"可见光图像{urgent_vis_coords}呈现{vis_feature}" if urgent_vis_coords else f"可见光图像{vis_feature}"
        answer_parts.append(
            f"其中，最需要优先处理的是{urgent_type}，在红外图像{urgent_ir_coords}的范围形成{ir_feature}，"
            f"而在{vis_desc}，{impact}。这需要优先处理，因为{reason}。"
        )

    # 其他缺陷描述
    other_faults = priority_faults[1:]
    if other_faults:
        answer_parts.append("其他异常情况的影响如下：")
        for fault_type, priority, ir_coords, vis_coords in other_faults:
            if fault_type in descriptions:
                ir_feature = descriptions[fault_type]["ir_feature"]
                vis_feature = descriptions[fault_type]["vis_feature"]
                impact = descriptions[fault_type]["impact"]
                vis_desc = f"可见光图像{vis_coords}呈现{vis_feature}" if vis_coords else f"可见光图像{vis_feature}"
                if priority <= 7:
                    desc = f"相较之下，{fault_type}在红外图像{ir_coords}的范围形成{ir_feature}，而在{vis_desc}，{impact}。"
                else:
                    desc = f"此外，{fault_type}在红外图像{ir_coords}的范围形成{ir_feature}，而在{vis_desc}，{impact}。"
                answer_parts.append(desc)

    return " ".join(answer_parts)

# 测试用例1：高优先级缺陷（组串短路+内部缺陷）
label1 = (
    "总共有2个故障；故障1：组串短路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)；"
    "故障2：内部缺陷，Bounding Box只在红外图像出现，坐标为(532, 24), (621, 38)"
)
print("测试用例1 输出：")
print(TypeOne_D4_Process_Priority(label1))

# 测试用例2：中度+低优先级缺陷（表面破损+草木遮挡+边缘积尘）
label2 = (
    "总共有3个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)"
)
print("\n测试用例2 输出：")
print(TypeOne_D4_Process_Priority(label2))

# 测试用例3：单一低优先级缺陷（边缘积尘）
label3 = (
    "总共有1个故障；故障1：边缘积尘，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)"
)
print("\n测试用例3 输出：")
print(TypeOne_D4_Process_Priority(label3))

测试用例1 输出：
对这组图像进行分析，共检测到2种异常情况：组串短路、内部缺陷。 其中，最需要优先处理的是组串短路，在红外图像坐标(557, 155)到(587, 165)的范围形成连续点状热斑，而在可见光图像无明显特征，会导致光伏面板部分或全部停发，因线路过热形成严重热斑，存在设备烧毁或火灾的潜在风险。这需要优先处理，因为安全风险最高，需要立即处理避免火灾发生。 其他异常情况的影响如下： 相较之下，内部缺陷在红外图像坐标(532, 24)到(621, 38)的范围形成点状热斑，而在可见光图像无明显特征，属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险。

测试用例2 输出：
对这组图像进行分析，共检测到3种异常情况：边缘积尘、草木遮挡、表面破损。 其中，最需要优先处理的是表面破损，在红外图像坐标(365, 148)到(421, 176)的范围形成区域热斑，而在可见光图像坐标(1399, 617)到(1598, 725)呈现破损现象，因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险。这需要优先处理，因为玻璃破碎后会进水、进尘，损坏内部电路，需尽快处理以防止故障扩大。。 其他异常情况的影响如下： 相较之下，草木遮挡在红外图像坐标(73, 81)到(110, 108)的范围形成区域热斑，而在可见光图像坐标(259, 345)到(289, 398)呈现草木，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率。 此外，边缘积尘在红外图像坐标(300, 400)到(350, 450)的范围形成区域热斑，而在可见光图像坐标(900, 1000)到(950, 1050)呈现边缘积尘，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。

测试用例3 输出：
对这组图像进行分析，共检测到1种异常情况：边缘积尘。 其中，最需要优先处理的是边缘积尘，在红外图像坐标(73, 81)到(110, 108)的范围形成区域热斑，而在可见光图像坐标(259, 345)到(289, 398)呈现边缘积尘，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。这需要优先处理，因为影响相对较小，可在常规清洁时顺带处理。


### TypeOne_id22
**根据这些照片显示的异常情况（热斑温差、遮挡面积、污染程度、潜在风险），哪些问题是需要最优先处理的？**

In [ ]:
def TypeOne_id22(label):
    if label == "没有缺陷":
        return "结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。"

    faults = parse_label(label)
    total_faults = len(faults)

    if total_faults == 0:
        return "结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。"

    answer_parts = [f"结合这组图像进行分析，共有{total_faults}处缺陷。其中，"]

    # 获取唯一的缺陷类型，包括仅红外的
    fault_types = list(set(f["type"] for f in faults))

    for fault_type in fault_types:
        instances = [f for f in faults if f['type'] == fault_type]
        ir_feature = descriptions.get(fault_type, {}).get('ir_feature', '未知热斑特征')
        vis_feature = descriptions.get(fault_type, {}).get('vis_feature', '未知异常')

        # 根据缺陷类型设置可见光描述
        if fault_type == "表面破损":
            vis_desc = "裂纹或破片等破损现象"
        elif fault_type == "草木遮挡":
            vis_desc = "树叶或草木等遮挡物"
        elif fault_type == "边缘积尘":
            vis_desc = "灰尘堆积"
        elif fault_type == "表面污迹":
            vis_desc = "污渍如鸟粪或泥土"
        elif fault_type == "固定设施遮挡":
            vis_desc = "固定设施或阴影"
        elif fault_type == "组件异位":
            vis_desc = "明显的太阳光反射"
        else:
            vis_desc = vis_feature

        ir_coords_list = [inst['ir_coords'] for inst in instances]
        vis_coords_list = [inst['vis_coords'] if inst['vis_coords'] else None for inst in instances]

        # 假设同类型缺陷的vis_coords一致性
        has_vis = instances[0]['vis_coords'] is not None

        # 如果无可见光特征，使用vis_feature
        if not has_vis:
            vis_desc = vis_feature  # 通常为"无明显特征"

        if len(ir_coords_list) > 1:
            ir_str = "、".join(ir_coords_list[:-1]) + "和" + ir_coords_list[-1]
            if has_vis:
                vis_str = "、".join(vis_coords_list[:-1]) + "和" + vis_coords_list[-1]
                answer_parts.append(f"多处{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光对应{vis_str}的区域，能看到{vis_desc}。")
            else:
                answer_parts.append(f"多处{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光区域无明显特征。")
        else:
            ir_str = ir_coords_list[0]
            if has_vis:
                vis_str = vis_coords_list[0]
                answer_parts.append(f"{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光对应{vis_str}的区域，能看到{vis_desc}。")
            else:
                answer_parts.append(f"{fault_type}缺陷，体现在热成像上{ir_str}的范围，呈现出{ir_feature}特征，在可见光上对应{vis_str}的区域。")

    return "".join(answer_parts)

print("测试用例1 输出：")
print(TypeOne_id22(label1))

print("\n测试用例2 输出：")
print(TypeOne_id22(label2))

print("\n测试用例3 输出：")
print(TypeOne_id22(label3))

print("测试用例4 输出：")
print(TypeOne_id22(label4))

测试用例1 输出：
结合这组图像进行分析，共有1处缺陷。其中，表面破损缺陷，体现在热成像上坐标(365, 148)到(421, 176)的范围，呈现出区域热斑特征，在可见光对应坐标(1399, 617)到(1598, 725)的区域，能看到裂纹或破片等破损现象。

测试用例2 输出：
结合这组图像进行分析，共有3处缺陷。其中，草木遮挡缺陷，体现在热成像上坐标(73, 81)到(110, 108)的范围，呈现出区域热斑特征，在可见光对应坐标(259, 345)到(289, 398)的区域，能看到树叶或草木等遮挡物。边缘积尘缺陷，体现在热成像上坐标(200, 300)到(250, 350)的范围，呈现出区域热斑特征，在可见光对应坐标(800, 900)到(850, 950)的区域，能看到灰尘堆积。组串开路缺陷，体现在热成像上坐标(557, 155)到(587, 165)的范围，呈现出整体热斑特征，在可见光区域无明显特征。

测试用例3 输出：
结合这组图像进行分析，共有5处缺陷。其中，边缘积尘缺陷，体现在热成像上坐标(300, 400)到(350, 450)的范围，呈现出区域热斑特征，在可见光对应坐标(900, 1000)到(950, 1050)的区域，能看到灰尘堆积。多处草木遮挡缺陷，体现在热成像上坐标(73, 81)到(110, 108)和坐标(200, 300)到(250, 350)的范围，呈现出区域热斑特征，在可见光对应坐标(259, 345)到(289, 398)和坐标(800, 900)到(850, 950)的区域，能看到树叶或草木等遮挡物。组件异位缺陷，体现在热成像上坐标(557, 155)到(587, 165)的范围，呈现出大面积区域热斑特征，在可见光区域无明显特征。表面破损缺陷，体现在热成像上坐标(365, 148)到(421, 176)的范围，呈现出区域热斑特征，在可见光对应坐标(1399, 617)到(1598, 725)的区域，能看到裂纹或破片等破损现象。
测试用例4 输出：
结合这组图像进行分析，没有缺陷。在这个热斑位置的可见光照片上，没有看到明显的异常，如裂纹、破片、污渍、遮挡物或阴影。


### TypeOne_Wire_Defect
**接线问题**

In [ ]:
def TypeOne_D1_Hotpot_Type(label):
    if label == "没有缺陷":
        return "结合这组图像进行分析，没有发现明显缺陷。"

    faults = parse_label(label)
    total_faults = len(faults)

    if total_faults == 0:
        return "结合这组图像进行分析，没有发现明显缺陷。"

    answer_parts = [f"结合这组图像进行分析，共有{total_faults}个缺陷。"]

    # 获取唯一的缺陷类型
    fault_types = list(set(f["type"] for f in faults))

    wiring_faults = []
    non_wiring_faults = []

    for fault_type in fault_types:
        if "线路问题" in descriptions.get(fault_type, {}).get('cause', ''):
            wiring_faults.append(fault_type)
        else:
            non_wiring_faults.append(fault_type)

    if not wiring_faults:
        answer_parts.append("这些缺陷不属于接线问题。")
    else:
        answer_parts.append("其中，可能属于接线问题的缺陷包括：")
        for fault_type in wiring_faults:
            instances = [f for f in faults if f['type'] == fault_type]
            cause = descriptions.get(fault_type, {}).get('cause', "")
            ir_feature = descriptions.get(fault_type, {}).get('ir_feature', '未知热斑特征')
            ir_coords_list = [inst['ir_coords'].replace('坐标', '') for inst in instances]

            if len(ir_coords_list) > 1:
                ir_str = "、".join(ir_coords_list[:-1]) + "和" + ir_coords_list[-1]
                answer_parts.append(f"多处{fault_type}，体现在热成像上{ir_str}的范围，呈现出{ir_feature}，其{cause}")
            else:
                ir_str = ir_coords_list[0]
                answer_parts.append(f"{fault_type}，体现在热成像上{ir_str}的范围，呈现出{ir_feature}，其{cause}")

        if non_wiring_faults:
            answer_parts.append("其他缺陷如")
            for fault_type in non_wiring_faults:
                instances = [f for f in faults if f['type'] == fault_type]
                cause = descriptions.get(fault_type, {}).get('cause', "")
                ir_feature = descriptions.get(fault_type, {}).get('ir_feature', '未知热斑特征')
                ir_coords_list = [inst['ir_coords'].replace('坐标', '') for inst in instances]

                if len(ir_coords_list) > 1:
                    ir_str = "、".join(ir_coords_list[:-1]) + "和" + ir_coords_list[-1]
                    answer_parts.append(f"多处{fault_type}，体现在热成像上{ir_str}的范围，呈现出{ir_feature}，其{cause}；")
                else:
                    ir_str = ir_coords_list[0]
                    answer_parts.append(f"{fault_type}，体现在热成像上{ir_str}的范围，呈现出{ir_feature}，其{cause}；")
            # 移除最后一个分号
            answer_parts[-1] = answer_parts[-1].rstrip('；') + "不属于接线问题。"

    return "".join(answer_parts)

# 测试用例1
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1:", TypeOne_D1_Hotpot_Type(label1))

# 测试用例2
label2 = (
    "总共有3个故障；故障1：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("测试用例2:", TypeOne_D1_Hotpot_Type(label2))

# 测试用例3
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：边缘积尘，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("测试用例3:", TypeOne_D1_Hotpot_Type(label3))

# 测试用例4
label4 = (
    "没有缺陷"
)
print("测试用例4:", TypeOne_D1_Hotpot_Type(label4))

测试用例1: 结合这组图像进行分析，共有1个缺陷。这些缺陷不属于接线问题。
测试用例2: 结合这组图像进行分析，共有3个缺陷。这些缺陷不属于接线问题。
测试用例3: 结合这组图像进行分析，共有5个缺陷。这些缺陷不属于接线问题。
测试用例4: 结合这组图像进行分析，没有发现明显缺陷。


### TypeOne_D1_Hotpot_Type
**红外图像中某区域显示高温异常，具体可能是什么故障？（区分热斑、接线故障或灰尘遮挡）**

In [ ]:
def TypeOne_D1_Hotpot_Type(label):
    if label == "没有缺陷":
        return "结合这组图像进行分析，在红外图像中未显示高温异常，没有发现明显缺陷。"

    faults = parse_label(label)
    total_faults = len(faults)
    if total_faults == 0:
        return "结合这组图像进行分析，在红外图像中未显示高温异常，没有发现明显缺陷。"

    answer_parts = [f"结合这组图像进行分析，共有{total_faults}处缺陷，可能导致红外图像中对应区域显示高温异常。"]

    # 原映射（保留以兜底）
    categories = {
        "热斑类故障": ["内部缺陷", "二极管故障"],
        "接线故障": ["组串短路", "组串开路"],
        "灰尘或遮挡类故障": ["草木遮挡", "固定设施遮挡", "表面污迹", "边缘积尘"]
    }
    # 简洁类别名（用于成文落款）
    concise_cat = {
        "草木遮挡": "遮挡类故障",
        "固定设施遮挡": "遮挡类故障",
        "表面污迹": "灰尘类故障",
        "边缘积尘": "灰尘类故障",
        "表面破损": "表面损坏类故障",
        "内部缺陷": "热斑类故障",
        "二极管故障": "热斑类或接线类故障",
        "组串短路": "接线类故障",
        "组串开路": "接线类故障",
        "组件异位": "结构异位类故障",
    }

    fault_types = list(set(f["type"] for f in faults))
    num_word = lambda n: "1处" if n == 1 else ("2处" if n == 2 else f"{n}处")

    for fault_type in fault_types:
        instances = [f for f in faults if f['type'] == fault_type]
        ir_coords_list = [inst['ir_coords'].replace('坐标', '') for inst in instances]
        ir_str = "和".join(ir_coords_list) if len(ir_coords_list) > 1 else ir_coords_list[0]

        if fault_type == "二极管故障":
          ir_str = "和".join(ir_coords_list) if len(ir_coords_list) > 1 else ir_coords_list[0]
          answer_parts.append(
              f"{num_word(len(instances))}二极管故障出现在热成像上{ir_str}的范围，"
              f"呈现出条状热斑。该类故障可能由两类原因引起："
              f"一是外部线路存在虚接、过载等问题，导致二极管过流烧毁；"
              f"二是内置二极管自身老化或制造缺陷所致。"
              f"前者属于接线类故障，后者属于热斑类故障。"
          )
          continue

        ir_feature = descriptions.get(fault_type, {}).get('ir_feature', '未知热斑特征')
        cause_raw = descriptions.get(fault_type, {}).get('cause', '')

        cause_core = cause_raw.lstrip("其").lstrip("由于").strip("，。 　")
        cause_phrase = f"其是由于{cause_core}" if cause_core else ""

        fallback_cat = "其他故障"
        for cat, types in categories.items():
            if fault_type in types:
                fallback_cat = cat
                break
        cat_text = concise_cat.get(fault_type, fallback_cat)


        # 成文模板：数量 + 名称 + 出现位置 + 形态 + 原因 + 类别
        answer_parts.append(
            f"{num_word(len(instances))}{fault_type}出现在热成像上{ir_str}的范围，呈现出{ir_feature}，{cause_phrase}，属于{cat_text}。"
            .replace("，，", "，").replace("。，", "。")
        )

    return "".join(answer_parts)


# 测试用例1
label1 = (
    "总共有1个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)"
)
print("测试用例1:", TypeOne_D1_Hotpot_Type(label1))

# 测试用例2
label2 = (
    "总共有3个故障；故障1：二极管故障，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障2：边缘积尘，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障3：组串开路，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("测试用例2:", TypeOne_D1_Hotpot_Type(label2))

# 测试用例3
label3 = (
    "总共有5个故障；故障1：表面破损，Bounding Box在红外图像的坐标为(365, 148), (421, 176)，在可见光图像的坐标为(1399, 617), (1598, 725)；"
    "故障2：草木遮挡，Bounding Box在红外图像的坐标为(73, 81), (110, 108)，在可见光图像的坐标为(259, 345), (289, 398)；"
    "故障3：草木遮挡，Bounding Box在红外图像的坐标为(200, 300), (250, 350)，在可见光图像的坐标为(800, 900), (850, 950)；"
    "故障4：草木遮挡，Bounding Box在红外图像的坐标为(300, 400), (350, 450)，在可见光图像的坐标为(900, 1000), (950, 1050)；"
    "故障5：内部缺陷，Bounding Box只在红外图像出现，坐标为(557, 155), (587, 165)"
)
print("测试用例3:", TypeOne_D1_Hotpot_Type(label3))

# 测试用例4
label4 = (
    "没有缺陷"
)
print("测试用例4:", TypeOne_D1_Hotpot_Type(label4))

测试用例1: 结合这组图像进行分析，共有1处缺陷，可能导致红外图像中对应区域显示高温异常。1处表面破损出现在热成像上(365, 148)到(421, 176)的范围，呈现出区域热斑，其是由于外部冲击、冰雹、动物啃咬等造成玻璃或背板破损，属于表面损坏类故障。
测试用例2: 结合这组图像进行分析，共有3处缺陷，可能导致红外图像中对应区域显示高温异常。1处二极管故障出现在热成像上(73, 81)到(110, 108)的范围，呈现出条状热斑。该类故障可能由两类原因引起：一是外部线路存在虚接、过载等问题，导致二极管过流烧毁；二是内置二极管自身老化或制造缺陷所致。前者属于接线类故障，后者属于热斑类故障。1处组串开路出现在热成像上(557, 155)到(587, 165)的范围，呈现出整体热斑，其是由于主要因线路接头松动/脱落、线缆断裂、接线端子氧化接触不良导致，属于接线类故障。1处边缘积尘出现在热成像上(200, 300)到(250, 350)的范围，呈现出区域热斑，其是由于灰尘在组件边缘积累，通常因清洁不当或环境尘土多导致，属于灰尘类故障。
测试用例3: 结合这组图像进行分析，共有5处缺陷，可能导致红外图像中对应区域显示高温异常。1处内部缺陷出现在热成像上(557, 155)到(587, 165)的范围，呈现出点状热斑，其是由于如电池片隐裂、断栅、焊带脱落等，通常由于制造缺陷、安装不当或环境因素引起，属于热斑类故障。1处表面破损出现在热成像上(365, 148)到(421, 176)的范围，呈现出区域热斑，其是由于外部冲击、冰雹、动物啃咬等造成玻璃或背板破损，属于表面损坏类故障。3处草木遮挡出现在热成像上(73, 81)到(110, 108)和(200, 300)到(250, 350)和(300, 400)到(350, 450)的范围，呈现出区域热斑，其是由于周边植物生长未及时清理导致，属于遮挡类故障。
测试用例4: 结合这组图像进行分析，在红外图像中未显示高温异常，没有发现明显缺陷。


### TypeOne_D5_Hotpot_Position
**红外照片中显示的异常热点区域具体位置是哪里？**

In [ ]:
# 红外照片中显示的异常热点区域具体位置是哪里？
def TypeOne_D5_Hotpot_Position(label):
    f = [x for x in parse_label(label) if x.get('ir_coords')]
    if not f: return "对这组图像进行分析，未检测到红外照片中的异常热点区域，该光伏板没有明显缺陷"
    g, n = {}, len(f)
    for x in f: g.setdefault(x['type'], []).append(x['ir_coords'])
    num = lambda k: "一处" if k==1 else ("两处" if k==2 else f"{k}处")
    join = lambda L: L[0] if len(L)==1 else ("和".join(L) if len(L)==2 else "、".join(L[:-1])+"和"+L[-1])
    if n==1:
        t, L = next(iter(g.items()))
        return f"对这组图像进行分析，共检测到1处异常情况，为{t}，表现为{descriptions.get(t,{}).get('ir_feature','未知特征')}，位于红外图像的{L[0]}的范围。这些热点区域表明光伏组件存在潜在问题，需要进一步检查。"
    s=[f"有{num(len(L))}{t}，红外图像上表现为{descriptions.get(t,{}).get('ir_feature','未知特征')}，位于坐标{join(L)}的范围" for t,L in g.items()]
    return f"对这组图像进行分析，共检测到{n}处异常情况：" + "；".join(s) + "；这些热点区域表明光伏组件存在潜在问题，需要进一步检查。"

print("测试用例1 输出：")
print(TypeOne_D5_Hotpot_Position(label1))
print("\n测试用例2 输出：")
print(TypeOne_D5_Hotpot_Position(label2))
print("\n测试用例3 输出：")
print(TypeOne_D5_Hotpot_Position(label3))
print("测试用例4 输出：")
print(TypeOne_D5_Hotpot_Position(label4))

测试用例1 输出：
对这组图像进行分析，共检测到1处异常情况，为表面破损，表现为区域热斑，位于红外图像的坐标(365, 148)到(421, 176)的范围。这些热点区域表明光伏组件存在潜在问题，需要进一步检查。

测试用例2 输出：
对这组图像进行分析，共检测到3处异常情况：有一处草木遮挡，红外图像上表现为区域热斑，位于坐标坐标(73, 81)到(110, 108)的范围；有一处边缘积尘，红外图像上表现为区域热斑，位于坐标坐标(200, 300)到(250, 350)的范围；有一处组串开路，红外图像上表现为整体热斑，位于坐标坐标(557, 155)到(587, 165)的范围；这些热点区域表明光伏组件存在潜在问题，需要进一步检查。

测试用例3 输出：
对这组图像进行分析，共检测到5处异常情况：有一处表面破损，红外图像上表现为区域热斑，位于坐标坐标(365, 148)到(421, 176)的范围；有两处草木遮挡，红外图像上表现为区域热斑，位于坐标坐标(73, 81)到(110, 108)和坐标(200, 300)到(250, 350)的范围；有一处边缘积尘，红外图像上表现为区域热斑，位于坐标坐标(300, 400)到(350, 450)的范围；有一处内部缺陷，红外图像上表现为点状热斑，位于坐标坐标(557, 155)到(587, 165)的范围；这些热点区域表明光伏组件存在潜在问题，需要进一步检查。
测试用例4 输出：
对这组图像进行分析，未检测到红外照片中的异常热点区域，该光伏板没有明显缺陷


### TypeOne_D5_PV_Impact
**这个有问题的地方影响发电不？**

In [ ]:
# 这个有问题的地方影响发电不？
def generate_answer_impact_summary(label):
    f = parse_label(label)
    if not f: return "对这组图像进行分析，未检测到异常情况。结论：不影响发电。"
    # 聚合各缺陷的数量与严重度
    s = {}
    for x in f:
        t = x['type']; d = descriptions.get(t, {})
        s.setdefault(t, {"cnt":0, "p": d.get("peocess priority", 999), "impact": d.get("impact","可能影响发电效率"), "reason": d.get("reason","")})
        s[t]["cnt"] += 1
    types = sorted(s.items(), key=lambda kv: kv[1]["p"])
    T, N = len(types), len(f)
    top_name, top = types[0][0], types[0][1]
    intro = f"对这组图像进行分析，检测到{T}种问题，共{N}处。"
    core = f"会影响发电。影响最严重的是{top_name}，{top['impact']}。"
    if T == 1: return intro + core
    others = "、".join([f"{v['cnt']}处{t}" for t,v in types[1:][:2]])
    tail = f"其余问题包括{others}{'等' if T>3 else ''}，亦会不同程度降低发电效率，建议按优先级依次处理。"
    return intro + core + tail

def TypeOne_D5_PV_Impact(label):
    f = parse_label(label)
    if not f: return "对这组图像进行分析，未检测到异常情况，因此光伏组件发电不受影响。结论：不影响发电。"
    # 按缺陷类型聚合坐标（优先IR，无则VIS）
    g = {}
    for x in f:
        t = x['type']; c = x.get('ir_coords') or x.get('vis_coords')
        g.setdefault(t, []).append(c) if c else g.setdefault(t, [])
    join = lambda L: (L[0] if len(L)==1 else ("和".join(L) if len(L)==2 else "、".join(L[:-1])+"和"+L[-1]))
    T = len(g)

    # 只有一种问题
    if T == 1:
        (t, L) = next(iter(g.items()))
        loc = (f"出现在热成像{join(L)}的区域，" if L else "")
        return f"对这组图像进行分析，共检测到1种问题，为{t}，{loc}{descriptions.get(t,{}).get('impact','可能影响发电效率')}。结论：会影响发电，建议尽快处置以恢复发电水平。"

    # 多种问题（同类多处合并坐标）
    parts = []
    for t, L in g.items():
        loc = (f"出现在热成像{join(L)}的区域，" if L else "")
        parts.append(f"{t}，{loc}{descriptions.get(t,{}).get('impact','可能影响发电效率')}。")
    return "".join(parts) + generate_answer_impact_summary(label)

print("测试用例1 输出：")
print(TypeOne_D5_PV_Impact(label1))
print("\n测试用例2 输出：")
print(TypeOne_D5_PV_Impact(label2))
print("\n测试用例3 输出：")
print(TypeOne_D5_PV_Impact(label3))
print("测试用例4 输出：")
print(TypeOne_D5_PV_Impact(label4))

测试用例1 输出：
对这组图像进行分析，共检测到1种问题，为表面破损，出现在热成像坐标(365, 148)到(421, 176)的区域，因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险。结论：会影响发电，建议尽快处置以恢复发电水平。

测试用例2 输出：
草木遮挡，出现在热成像坐标(73, 81)到(110, 108)的区域，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率。边缘积尘，出现在热成像坐标(200, 300)到(250, 350)的区域，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。组串开路，出现在热成像坐标(557, 155)到(587, 165)的区域，会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率。对这组图像进行分析，检测到3种问题，共3处。会影响发电。影响最严重的是组串开路，会直接导致整个组串停止发电，形成大范围热斑，显著降低系统发电效率。其余问题包括1处草木遮挡、1处边缘积尘，亦会不同程度降低发电效率，建议按优先级依次处理。

测试用例3 输出：
表面破损，出现在热成像坐标(365, 148)到(421, 176)的区域，因破损导致透光率下降，可能导致发电效率下降10%-30%，存在进水损坏电路的潜在风险。草木遮挡，出现在热成像坐标(73, 81)到(110, 108)和坐标(200, 300)到(250, 350)的区域，会导致遮挡区域发电显著减少并影响周边组件，通常不会直接导致系统瘫痪，但遮挡面积会随植物生长扩大，快速降低局部发电效率。边缘积尘，出现在热成像坐标(300, 400)到(350, 450)的区域，因积尘量少，污染程度较低，通常引起的发电效率下降小于10%。内部缺陷，出现在热成像坐标(557, 155)到(587, 165)的区域，属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险。对这组图像进行分析，检测到4种问题，共5处。会影响发电。影响最严重的是内部缺陷，属于隐性故障，会降低发电效率下降10%-50%，而且会持续恶化，存在局部瘫痪的潜在风险。其余问题包括1处表面破损、2处草木遮挡等，亦会不同程度降低发电效率，建议按优先级依次处理。
测试用例4 输出：
对

### TypeOne_D1_Defect_Position
**存在缺陷的光伏组件的坐标位置是多少？**

In [ ]:
# 存在缺陷的光伏组件的坐标位置是多少？
def TypeOne_D1_Defect_Position(label):
    faults = parse_label(label)
    if not faults:
        return "对这组图像进行分析，未检测到明显异常情况，因此无缺陷光伏组件的坐标位置。"
    intro = f"对这组图像进行分析，共检测到{len(faults)}处异常情况："
    details = []
    for i, fault in enumerate(faults, 1):
        fault_type = fault['type']
        ir_coords = fault['ir_coords']
        vis_coords = fault['vis_coords']
        coord_desc = []
        if ir_coords:
            coord_desc.append(f"在红外图像上位于{ir_coords}的范围")
        if vis_coords:
            coord_desc.append(f"可见光图像上位于{vis_coords}的范围")
        if coord_desc:
            details.append(f"异常{i}为{fault_type}，{'并且'.join(coord_desc)}。")
        else:
            details.append(f"异常{i}为{fault_type}，无可用坐标信息。")
    return intro + "".join(details) + "这些坐标可用于定位并检修受影响的组件。"

print("测试用例1 输出：")
print(TypeOne_D1_Defect_Position(label1))
print("\n测试用例2 输出：")
print(TypeOne_D1_Defect_Position(label2))
print("\n测试用例3 输出：")
print(TypeOne_D1_Defect_Position(label3))
print("测试用例4 输出：")
print(TypeOne_D1_Defect_Position(label4))

测试用例1 输出：
对这组图像进行分析，共检测到1处异常情况：异常1为表面破损，在红外图像上位于坐标(365, 148)到(421, 176)的范围并且可见光图像上位于坐标(1399, 617)到(1598, 725)的范围。这些坐标可用于定位并检修受影响的组件。

测试用例2 输出：
对这组图像进行分析，共检测到3处异常情况：异常1为草木遮挡，在红外图像上位于坐标(73, 81)到(110, 108)的范围并且可见光图像上位于坐标(259, 345)到(289, 398)的范围。异常2为边缘积尘，在红外图像上位于坐标(200, 300)到(250, 350)的范围并且可见光图像上位于坐标(800, 900)到(850, 950)的范围。异常3为组串开路，在红外图像上位于坐标(557, 155)到(587, 165)的范围。这些坐标可用于定位并检修受影响的组件。

测试用例3 输出：
对这组图像进行分析，共检测到5处异常情况：异常1为表面破损，在红外图像上位于坐标(365, 148)到(421, 176)的范围并且可见光图像上位于坐标(1399, 617)到(1598, 725)的范围。异常2为草木遮挡，在红外图像上位于坐标(73, 81)到(110, 108)的范围并且可见光图像上位于坐标(259, 345)到(289, 398)的范围。异常3为草木遮挡，在红外图像上位于坐标(200, 300)到(250, 350)的范围并且可见光图像上位于坐标(800, 900)到(850, 950)的范围。异常4为边缘积尘，在红外图像上位于坐标(300, 400)到(350, 450)的范围并且可见光图像上位于坐标(900, 1000)到(950, 1050)的范围。异常5为内部缺陷，在红外图像上位于坐标(557, 155)到(587, 165)的范围。这些坐标可用于定位并检修受影响的组件。
测试用例4 输出：
对这组图像进行分析，未检测到明显异常情况，因此无缺陷光伏组件的坐标位置。


### TypeOne_D6_RGB_Defct_Visibility
**在这个热斑位置的可见光照片上，能看到什么明显的异常吗？比如裂纹、破片、污渍（鸟粪、泥土）、遮挡物（树叶、灰尘堆积、阴影）、烧毁痕迹、或者接线盒/连接器问题？**

In [ ]:
VIS_DESCRIPTIONS_FOR_Q = {
    "组串短路": {
        "ir_feature": "连续点状热斑",
        "rgb_sign": "通常无明显异常，因为此类问题多由电气短路引起，表面难以察觉",
        "visible": False,
        "reason": "组串短路多由接线或绝缘问题引起，肉眼无法直接识别"
    },
    "组串开路": {
        "ir_feature": "整体热斑",
        "rgb_sign": "一般无明显异常，外观正常",
        "visible": False,
        "reason": "组串开路多为线路连接问题"
    },
    "二极管故障": {
        "ir_feature": "条状热斑",
        "rgb_sign": "未见明显异常",
        "visible": False,
        "reason": "二极管故障位于接线盒内部，肉眼不可见"
    },
    "内部缺陷": {
        "ir_feature": "点状热斑",
        "rgb_sign": "未发现明显破裂或污染特征",
        "visible": False,
        "reason": "内部缺陷，如隐裂、断栅等，属于电池片内部隐性问题，通常外观呈现正常，表面难以察觉"
    },
    "组件异位": {
        "ir_feature": "大面积区域热斑",
        "rgb_sign": "组件角度异常或反光带，与邻近组件不对齐",
        "visible": True,
        "reason": "安装角度偏差导致"
    },
    "表面破损": {
        "ir_feature": "区域热斑",
        "rgb_sign": "明显的裂纹、破片或破碎纹理",
        "visible": True,
        "reason": "组件玻璃破裂导致透光性下降"
    },
    "草木遮挡": {
        "ir_feature": "区域热斑",
        "rgb_sign": "叶片、草木或枝条压覆在组件表面",
        "visible": True,
        "reason": "周围植物生长形成遮挡"
    },
    "固定设施遮挡": {
        "ir_feature": "区域热斑或阴影",
        "rgb_sign": "支架、栏杆或线缆投影遮挡板面",
        "visible": True,
        "reason": "固定结构遮挡入射光"
    },
    "表面污迹": {
        "ir_feature": "区域热斑",
        "rgb_sign": "鸟粪、泥土或水渍形成的脏污斑块",
        "visible": True,
        "reason": "表面积灰或污染物附着"
    },
    "边缘积尘": {
        "ir_feature": "区域热斑",
        "rgb_sign": "组件边缘形成条带状灰尘堆积",
        "visible": True,
        "reason": "灰尘积聚在组件边缘，透光率下降"
    },
}

def TypeOne_D6_RGB_Defct_Visibility(label, descriptions=VIS_DESCRIPTIONS_FOR_Q):
    """
    问题：在这个热斑位置的可见光照片上，能看到什么明显的异常吗？
    """
    faults = parse_label(label)
    if not faults:
        return "对这组图像的热斑进行核查，未检测到热斑区域，因此在可见光照片上也未发现明显异常。"

    from collections import defaultdict
    grouped = defaultdict(list)
    for f in faults:
        grouped[f["type"]].append(f)

    parts = []
    parts.append(f"对这组图像的热斑进行核查，共检测到{len(faults)}处异常。")

    # 针对每种缺陷类型生成自然描述
    for defect_type, lst in grouped.items():
        desc = descriptions.get(defect_type, {})
        ir_feat = desc.get("ir_feature", "热斑特征不详")
        rgb_feat = desc.get("rgb_sign", "")
        visible = desc.get("visible", False)
        reason = desc.get("reason", "")

        # 多处 vs 单处
        if len(lst) == 1:
            f = lst[0]
            ir_pos = f["ir_coords"]
            if visible and f.get("vis_coords"):
                parts.append(
                    f"在红外图像中，{ir_pos}的区域呈现{ir_feat}特征，"
                    f"在可见光图像的对应位置，可以看见{rgb_feat}，这属于{defect_type}的表现。"
                )
            else:
                parts.append(
                    f"在红外图像中，{ir_pos}的区域呈现{ir_feat}特征，"
                    f"但在可见光图像的对应位置未发现明显异常，这是因为{reason}。"
                )
        else:
            # 多个同类缺陷
            ir_list = [f["ir_coords"] for f in lst]
            ir_str = "、".join(ir_list)
            if desc.get("visible", False):
                parts.append(
                    f"在红外图像的{ir_str}范围，共出现{len(lst)}处{ir_feat}现象，"
                    f"在可见光图像的对应位置，可以清晰看到{rgb_feat}，这属于{defect_type}类缺陷的典型表现。"
                )
            else:
                parts.append(
                    f"在红外图像的{ir_str}范围，共出现{len(lst)}处{ir_feat}现象，"
                    f"但在可见光图像上未见明显特征，这是因为{reason}。"
                )

    # 结尾总结
    visible_types = [t for t, lst in grouped.items() if any(descriptions.get(t,{}).get("visible", False) and f.get("vis_coords") for f in lst)]
    invisible_types = [t for t in grouped.keys() if t not in visible_types]

    # 结尾总结：聚焦“热斑位置”的可见光表现
    if visible_types or invisible_types:
        if visible_types and invisible_types:
            parts.append(
                f"总体来看，这些热斑位置中，有的在可见光图像上能够直观看到明显异常，"
                f"主要包括{ '、'.join(visible_types) }；"
                f"而其余热斑位置在可见光上未呈现明显特征，"
                f"对应的类型包括{ '、'.join(invisible_types) }。"
            )
        elif visible_types:
            parts.append(
                f"总体来看，这些热斑位置在可见光图像上均能看到明显异常，"
                f"主要表现为{ '、'.join(visible_types) }类缺陷。"
            )
        elif invisible_types:
            parts.append(
                f"总体来看，这些热斑位置在可见光图像上未见直观异常，"
                f"涉及的缺陷类型包括{ '、'.join(invisible_types) }。"
            )

    return "".join(parts)


print("测试用例1 输出：")
print(TypeOne_D6_RGB_Defct_Visibility(label1))
print("\n测试用例2 输出：")
print(TypeOne_D6_RGB_Defct_Visibility(label2))
print("\n测试用例3 输出：")
print(TypeOne_D6_RGB_Defct_Visibility(label3))
print("测试用例4 输出：")
print(TypeOne_D6_RGB_Defct_Visibility(label4))

测试用例1 输出：
对这组图像的热斑进行核查，共检测到1处异常。在红外图像中，坐标(365, 148)到(421, 176)的区域呈现区域热斑特征，在可见光图像的对应位置，可以看见明显的裂纹、破片或破碎纹理，这属于表面破损的表现。总体来看，这些热斑位置在可见光图像上均能看到明显异常，主要表现为表面破损类缺陷。

测试用例2 输出：
对这组图像的热斑进行核查，共检测到3处异常。在红外图像中，坐标(73, 81)到(110, 108)的区域呈现条状热斑特征，但在可见光图像的对应位置未发现明显异常，这是因为二极管故障位于接线盒内部，肉眼不可见。在红外图像中，坐标(200, 300)到(250, 350)的区域呈现区域热斑特征，在可见光图像的对应位置，可以看见组件边缘形成条带状灰尘堆积，这属于边缘积尘的表现。在红外图像中，坐标(557, 155)到(587, 165)的区域呈现整体热斑特征，但在可见光图像的对应位置未发现明显异常，这是因为组串开路多为线路连接问题。总体来看，这些热斑位置中，有的在可见光图像上能够直观看到明显异常，主要包括边缘积尘；而其余热斑位置在可见光上未呈现明显特征，对应的类型包括二极管故障、组串开路。

测试用例3 输出：
对这组图像的热斑进行核查，共检测到5处异常。在红外图像中，坐标(365, 148)到(421, 176)的区域呈现区域热斑特征，在可见光图像的对应位置，可以看见明显的裂纹、破片或破碎纹理，这属于表面破损的表现。在红外图像的坐标(73, 81)到(110, 108)、坐标(200, 300)到(250, 350)、坐标(300, 400)到(350, 450)范围，共出现3处区域热斑现象，在可见光图像的对应位置，可以清晰看到叶片、草木或枝条压覆在组件表面，这属于草木遮挡类缺陷的典型表现。在红外图像中，坐标(557, 155)到(587, 165)的区域呈现点状热斑特征，但在可见光图像的对应位置未发现明显异常，这是因为内部缺陷，如隐裂、断栅等，属于电池片内部隐性问题，通常外观呈现正常，表面难以察觉。总体来看，这些热斑位置中，有的在可见光图像上能够直观看到明显异常，主要包括表面破损、草木遮挡；而其余热斑位置在可见光上未呈现明显特征，对应的类型包括内部缺陷。
测试用例4 输出：
对这组图像的热斑进行核查，未检测到热斑区域，因此在可见光照片上也

### TypeOne_D6_RGB_Shadow_Impact
**可见光里光伏板表面的阴影，会让红外照片出现温度异常吗？**

In [ ]:
SHADOW_DESCRIPTIONS_FOR_Q = {
    "草木遮挡": {
        "ir_feature": "区域热斑，通常温度较周围偏高",
        "vis_feature": "可见光上可见树叶、枝条或杂草形成明显阴影",
        "effect": "遮挡区域因光照减少，电流失衡导致局部过热，红外图像上表现为热斑",
        "visible": True,
    },
    "固定设施遮挡": {
        "ir_feature": "区域热斑或阴影边缘温差明显",
        "vis_feature": "可见光上可见支架、栏杆或线缆投下固定阴影",
        "effect": "遮挡位置光照不足，组件受光不均，产生温度梯度，在红外图像上可能出现轻微温差",
        "visible": True,
    },
    "边缘积尘": {
        "ir_feature": "区域热斑（多集中于边缘）",
        "vis_feature": "可见光上边缘灰尘造成轻微阴影",
        "effect": "透光率下降导致边缘温度略高，产生细弱热斑",
        "visible": True,
    },
    "表面污迹": {
        "ir_feature": "局部弱热斑",
        "vis_feature": "可见光上可见污渍或鸟粪投影",
        "effect": "污迹遮光使电流不均，局部温度升高，在红外图像上表现为轻微热斑",
        "visible": True,
    },
    # 其他类型不会造成阴影型热斑
}
def TypeOne_D6_RGB_Shadow_Impact(label, descriptions=SHADOW_DESCRIPTIONS_FOR_Q):
    """
    问题：可见光里光伏板表面的阴影，会让红外照片出现温度异常吗？
    """
    faults = parse_label(label)
    if not faults:
        # 无任何异常 → 只保留原理解释 + 无阴影结论
        return (
            "从原理上看，光伏组件表面的阴影会导致受光区域电流分布不均，"
            "局部电池片因电流回流产生功率损耗，从而在红外图像中形成温度偏高的热斑。"
            "但在当前这组图像中，可见光照片未见明显阴影或遮挡现象，红外图像中也未出现对应的温度异常。"
        )

    from collections import defaultdict
    grouped = defaultdict(list)
    for f in faults:
        grouped[f["type"]].append(f)

    # 分类：与阴影相关 vs 无关
    shadow_related, non_shadow = [], []
    for defect_type, lst in grouped.items():
        if defect_type in descriptions:
            shadow_related.append((defect_type, lst, descriptions[defect_type]))
        else:
            non_shadow.append((defect_type, lst))

    # 开头原理说明
    parts = [
        "光伏组件表面的阴影会影响入射光照分布，使被遮挡区域电流减弱、局部过热，"
        "因此在红外图像中通常会表现为温度异常或热斑。这类现象在巡检中较为常见，"
        "尤其是由草木、支架、线缆等遮挡造成的局部阴影。"
    ]

    # 若有遮挡类缺陷，结合具体热斑位置说明
    if shadow_related:
        for defect_type, lst, d in shadow_related:
            ir_list = [f["ir_coords"] for f in lst]
            ir_str = "、".join(ir_list)
            parts.append(
                f"在当前图像中，红外图像的{ir_str}区域呈现{d['ir_feature']}，"
                f"而可见光图像在对应位置可见{d['vis_feature']}，"
                f"说明该处阴影确实引起了局部温度异常，这是{defect_type}类现象的典型表现。"
            )

    # 若无阴影类缺陷，说明未见此类温度异常
    else:
        # 检查是否存在其他类型缺陷（例如表面破损、内部缺陷等）
        other_types = [f["type"] for f in faults if f["type"] not in descriptions]
        if other_types:
            other_str = "、".join(sorted(set(other_types)))
            parts.append(
                f"结合当前这组图像分析，可见光图像上未发现明显的阴影或遮挡物，"
                f"红外图像中也未出现由遮挡导致的温度异常，说明该组件受光均匀，不存在阴影遮挡类异常。"
                f"此外，图像中检测到的其他异常类型（如{other_str}）与遮挡无关，不会引起阴影导致的温度差异。"
            )
        else:
            parts.append(
                "结合当前这组图像分析，可见光图像上未发现明显的阴影或遮挡物，"
                "红外图像中也未出现由遮挡导致的温度异常，说明该组件受光均匀，不存在阴影遮挡类异常。"
            )

    # 总结归纳
    shadow_types = "、".join([s[0] for s in shadow_related])
    other_types = [f["type"] for f in faults if f["type"] not in descriptions]
    if shadow_related:
        if other_types:
            other_str = "、".join(sorted(set(other_types)))
            parts.append(
                f"总体来看，本组图像中部分热斑与遮挡密切相关，"
                f"主要包括{shadow_types}等情况，可见光中的阴影确实在红外图像中对应了温度异常。"
                f"此外，还检测到{other_str}等非遮挡类异常，但这些现象与阴影无关，未对红外温度分布造成明显影响。"
            )
        else:
            parts.append(
                f"总体来看，本组图像中部分热斑与遮挡密切相关，"
                f"主要包括{shadow_types}等情况，可见光中的阴影确实在红外图像中对应了温度异常。"
            )
    else:
        if other_types:
            parts.append(
                f"总体来看，本组图像的热斑位置在可见光中未发现阴影，"
                f"红外图像中也未出现由遮挡引起的温度偏差；"
                f"仅检测到{ '、'.join(sorted(set(other_types))) }等非遮挡类异常，其与可见光表面阴影无关。"
            )
        else:
            parts.append(
                "总体来看，本组图像的热斑位置在可见光中未发现阴影，"
                "红外图像中也未出现由遮挡引起的温度偏差。"
            )

    return " ".join(parts)

print("测试用例1 输出：")
print(TypeOne_D6_RGB_Shadow_Impact(label1))
print("\n测试用例2 输出：")
print(TypeOne_D6_RGB_Shadow_Impact(label2))
print("\n测试用例3 输出：")
print(TypeOne_D6_RGB_Shadow_Impact(label3))
print("测试用例4 输出：")
print(TypeOne_D6_RGB_Shadow_Impact(label4))


测试用例1 输出：
光伏组件表面的阴影会影响入射光照分布，使被遮挡区域电流减弱、局部过热，因此在红外图像中通常会表现为温度异常或热斑。这类现象在巡检中较为常见，尤其是由草木、支架、线缆等遮挡造成的局部阴影。 结合当前这组图像分析，可见光图像上未发现明显的阴影或遮挡物，红外图像中也未出现由遮挡导致的温度异常，说明该组件受光均匀，不存在阴影遮挡类异常。此外，图像中检测到的其他异常类型（如表面破损）与遮挡无关，不会引起阴影导致的温度差异。 总体来看，本组图像的热斑位置在可见光中未发现阴影，红外图像中也未出现由遮挡引起的温度偏差；仅检测到表面破损等非遮挡类异常，其与可见光表面阴影无关。

测试用例2 输出：
光伏组件表面的阴影会影响入射光照分布，使被遮挡区域电流减弱、局部过热，因此在红外图像中通常会表现为温度异常或热斑。这类现象在巡检中较为常见，尤其是由草木、支架、线缆等遮挡造成的局部阴影。 在当前图像中，红外图像的坐标(200, 300)到(250, 350)区域呈现区域热斑（多集中于边缘），而可见光图像在对应位置可见可见光上边缘灰尘造成轻微阴影，说明该处阴影确实引起了局部温度异常，这是边缘积尘类现象的典型表现。 总体来看，本组图像中部分热斑与遮挡密切相关，主要包括边缘积尘等情况，可见光中的阴影确实在红外图像中对应了温度异常。此外，还检测到二极管故障、组串开路等非遮挡类异常，但这些现象与阴影无关，未对红外温度分布造成明显影响。

测试用例3 输出：
光伏组件表面的阴影会影响入射光照分布，使被遮挡区域电流减弱、局部过热，因此在红外图像中通常会表现为温度异常或热斑。这类现象在巡检中较为常见，尤其是由草木、支架、线缆等遮挡造成的局部阴影。 在当前图像中，红外图像的坐标(73, 81)到(110, 108)、坐标(200, 300)到(250, 350)、坐标(300, 400)到(350, 450)区域呈现区域热斑，通常温度较周围偏高，而可见光图像在对应位置可见可见光上可见树叶、枝条或杂草形成明显阴影，说明该处阴影确实引起了局部温度异常，这是草木遮挡类现象的典型表现。 总体来看，本组图像中部分热斑与遮挡密切相关，主要包括草木遮挡等情况，可见光中的阴影确实在红外图像中对应了温度异常。此外，还检测到内部缺陷、表面破损等非遮挡类异常，但这些现象与阴影无关，未对红外温度分布造成明显影响

### TypeOne_D6_RGB_Spot_Region
**红外照片上这些亮斑对应的位置，在可见光照片里是光伏板的哪个具体区域呀？**

In [ ]:
IR2RGB_DESCRIPTIONS_FOR_Q = {
    "内部缺陷": {
        "ir_feature": "点状亮斑",
        "rgb_feature": "可见光上通常外观正常",
        "reason": "属于电池片内部隐性问题，难以通过外观直接判断"
    },
    "表面破损": {
        "ir_feature": "区域亮斑",
        "rgb_feature": "表面可见裂纹或破片，玻璃破裂导致受光异常",
        "reason": "破损造成透光率下降，局部发热"
    },
    "草木遮挡": {
        "ir_feature": "区域亮斑",
        "rgb_feature": "表面可见叶片、枝条遮挡组件表面",
        "reason": "遮挡光照引起受光不均，局部温度升高"
    },
    "固定设施遮挡": {
        "ir_feature": "区域亮斑或阴影",
        "rgb_feature": "表面可见支架或线缆的阴影覆盖",
        "reason": "遮挡形成局部光照差异"
    },
    "边缘积尘": {
        "ir_feature": "边缘亮区",
        "rgb_feature": "可见灰尘堆积在组件边缘",
        "reason": "灰尘导致透光下降，形成局部升温"
    },
    "表面污迹": {
        "ir_feature": "局部亮斑，分布不规则",
        "rgb_feature": "可见表面脏污或鸟粪覆盖区域",
        "reason": "局部遮光使电流不均"
    },
    "组件异位": {
        "ir_feature": "大面积亮斑",
        "rgb_feature": "组件角度或安装位置偏移，形成异常反光",
        "reason": "倾角异常引起受光不均"
    },
    "组串短路": {
        "ir_feature": "连续点状亮斑",
        "rgb_feature": "可见光上无明显特征",
        "reason": "多由电气线路问题引起，表面难以辨识"
    },
    "组串开路": {
        "ir_feature": "大面积亮区",
        "rgb_feature": "可见光上无直观变化",
        "reason": "线路断开引起的电流中断"
    },
    "二极管故障": {
        "ir_feature": "条带状亮斑",
        "rgb_feature": "可见光上无直观变化",
        "reason": "接线盒内部故障不可外观识别"
    },
}

def TypeOne_D6_RGB_Spot_Region(label, descriptions=IR2RGB_DESCRIPTIONS_FOR_Q):
    """
    红外照片上这些亮斑对应的位置，在可见光照片里是光伏板的哪个具体区域呀？
    """
    faults = parse_label(label)
    if not faults:
        return "对这组图像进行分析，红外图像中未发现明显亮斑，因此在可见光照片中也无对应区域可定位。"

    from collections import defaultdict
    grouped = defaultdict(list)
    for f in faults:
        grouped[f["type"]].append(f)

    parts = [f"对这组图像进行分析，共识别到 {len(faults)} 处红外亮斑。"]

    for defect_type, lst in grouped.items():
        desc = descriptions.get(defect_type, {})
        ir_feat = desc.get("ir_feature", "亮斑特征不明")
        rgb_feat = desc.get("rgb_feature", "无明显可见光特征")
        reason = desc.get("reason", "表面特征不显著")

        # 所有红外坐标
        ir_coords = [f["ir_coords"] for f in lst if f.get("ir_coords")]
        ir_str = "、".join(ir_coords)

        # 所有可见光坐标
        vis_coords = [f["vis_coords"] for f in lst if f.get("vis_coords")]
        if vis_coords:
            vis_str = "、".join(vis_coords)
            parts.append(
                f"其中在红外图像的{ir_str}可见{ir_feat}；"
                f"对应地在可见光图像{vis_str}附近，"
                f"{rgb_feat}，呈现出与“{defect_type}”一致的特征。"
            )
        else:
            parts.append(
                f"其中在红外图像的{ir_str}可见{ir_feat}；"
                f"但在可见光图像中未发现对应区域的明显表征，"
                f"主要原因在于“{defect_type}”的特性：{reason}，"
                f"因此此类亮斑仅能依靠红外坐标进行定位与分析。"
            )

    # 总结归纳
    visible_cnt = sum(1 for f in faults if f.get("vis_coords"))
    invisible_cnt = len(faults) - visible_cnt
    if visible_cnt and invisible_cnt:
        parts.append(
            f"总体来看，这些红外亮斑中，大部分可在可见光图像中定位到组件表面具体区域，"
            f"表征清晰；少数亮斑因缺陷属于内部或电气问题，在可见光上难以识别，"
            f"需结合红外定位与机理分析进行判读。"
        )
    elif visible_cnt:
        parts.append(
            "总体来看，这些红外亮斑均能在可见光图像中准确对应到组件表面具体区域，"
            "红外与可见光具有良好的空间对应关系。"
        )
    else:
        parts.append(
            "总体来看，这些红外亮斑在可见光图像中均无明显对应区域，"
            "需依托红外坐标进行定位并结合缺陷机理进行分析。"
        )

    return " ".join(parts)



print("测试用例1 输出：")
print(TypeOne_D6_RGB_Spot_Region(label1))
print("\n测试用例2 输出：")
print(TypeOne_D6_RGB_Spot_Region(label2))
print("\n测试用例3 输出：")
print(TypeOne_D6_RGB_Spot_Region(label3))
print("测试用例4 输出：")
print(TypeOne_D6_RGB_Spot_Region(label4))



测试用例1 输出：
对这组图像进行分析，共识别到 1 处红外亮斑。 其中在红外图像的坐标(365, 148)到(421, 176)可见区域亮斑；对应地在可见光图像坐标(1399, 617)到(1598, 725)附近，表面可见裂纹或破片，玻璃破裂导致受光异常，呈现出与“表面破损”一致的特征。 总体来看，这些红外亮斑均能在可见光图像中准确对应到组件表面具体区域，红外与可见光具有良好的空间对应关系。

测试用例2 输出：
对这组图像进行分析，共识别到 3 处红外亮斑。 其中在红外图像的坐标(73, 81)到(110, 108)可见区域亮斑；对应地在可见光图像坐标(259, 345)到(289, 398)附近，表面可见叶片、枝条遮挡组件表面，呈现出与“草木遮挡”一致的特征。 其中在红外图像的坐标(200, 300)到(250, 350)可见边缘亮区；对应地在可见光图像坐标(800, 900)到(850, 950)附近，可见灰尘堆积在组件边缘，呈现出与“边缘积尘”一致的特征。 其中在红外图像的坐标(557, 155)到(587, 165)可见大面积亮区；但在可见光图像中未发现对应区域的明显表征，主要原因在于“组串开路”的特性：线路断开引起的电流中断，因此此类亮斑仅能依靠红外坐标进行定位与分析。 总体来看，这些红外亮斑中，大部分可在可见光图像中定位到组件表面具体区域，表征清晰；少数亮斑因缺陷属于内部或电气问题，在可见光上难以识别，需结合红外定位与机理分析进行判读。

测试用例3 输出：
对这组图像进行分析，共识别到 5 处红外亮斑。 其中在红外图像的坐标(365, 148)到(421, 176)可见区域亮斑；对应地在可见光图像坐标(1399, 617)到(1598, 725)附近，表面可见裂纹或破片，玻璃破裂导致受光异常，呈现出与“表面破损”一致的特征。 其中在红外图像的坐标(73, 81)到(110, 108)、坐标(200, 300)到(250, 350)可见区域亮斑；对应地在可见光图像坐标(259, 345)到(289, 398)、坐标(800, 900)到(850, 950)附近，表面可见叶片、枝条遮挡组件表面，呈现出与“草木遮挡”一致的特征。 其中在红外图像的坐标(300, 400)到(350, 450)可见边缘亮区；对应地在可见光图像坐标(900, 1000)到(

### TypeOne_D5_Pv_dot_Visibility
**要是光伏板有隐裂啥的，这两种照片能看出来不？**

In [ ]:
def TypeOne_D5_Pv_dot_Visibility(label):
    """
    问题：要是光伏板有隐裂啥的，这两种照片能看出来不？
    输入：原始 label 字符串
    输出：自然连贯、专业简明的回答
    """
    faults = parse_label(label)
    internal = [f for f in faults if f.get("type") == "内部缺陷"]

    # ===== ① 原理说明 =====
    intro = (
        "隐裂属于电池片内部结构性缺陷，通常发生在焊带应力或外力冲击后，"
        "导致电流通路受阻，形成局部电流不均。其显著特征是："
        "在红外图像中可见细小的点状光斑，"
        "并且由于裂纹位于电池片或封装层内部，表面多保持完整，"
        "因此可见光图像中难以直接识别。"
    )

    # ===== ② 结合本组图像 =====
    if not internal:
        # 若无内部缺陷
        return (
            intro
            + "结合当前这组图像进行分析，未发现符合隐裂特征的内部缺陷。"
        )

    # 若存在内部缺陷
    ir_locs = "、".join([f["ir_coords"] for f in internal if f.get("ir_coords")])
    return (
        intro
        + f" 在当前这组图像中，红外图像于{ir_locs}区域可见隐裂典型的点状热斑，"
        "表明该处电池片内部存在电流受阻与局部升温；"
        "而在可见光图像的对应区域表面未见明显特征，"
        "符合隐裂类缺陷‘红外可见、可见光难以察觉’的典型表现。"
    )


print("测试用例1 输出：")
print(TypeOne_D5_Pv_dot_Visibility(label1))
print("\n测试用例2 输出：")
print(TypeOne_D5_Pv_dot_Visibility(label2))
print("\n测试用例3 输出：")
print(TypeOne_D5_Pv_dot_Visibility(label3))
print("测试用例4 输出：")
print(TypeOne_D5_Pv_dot_Visibility(label4))

测试用例1 输出：
隐裂属于电池片内部结构性缺陷，通常发生在焊带应力或外力冲击后，导致电流通路受阻，形成局部电流不均。其显著特征是：在红外图像中可见细小的点状光斑，并且由于裂纹位于电池片或封装层内部，表面多保持完整，因此可见光图像中难以直接识别。结合当前这组图像进行分析，未发现符合隐裂特征的内部缺陷。

测试用例2 输出：
隐裂属于电池片内部结构性缺陷，通常发生在焊带应力或外力冲击后，导致电流通路受阻，形成局部电流不均。其显著特征是：在红外图像中可见细小的点状光斑，并且由于裂纹位于电池片或封装层内部，表面多保持完整，因此可见光图像中难以直接识别。结合当前这组图像进行分析，未发现符合隐裂特征的内部缺陷。

测试用例3 输出：
隐裂属于电池片内部结构性缺陷，通常发生在焊带应力或外力冲击后，导致电流通路受阻，形成局部电流不均。其显著特征是：在红外图像中可见细小的点状光斑，并且由于裂纹位于电池片或封装层内部，表面多保持完整，因此可见光图像中难以直接识别。 在当前这组图像中，红外图像于坐标(557, 155)到(587, 165)区域可见隐裂典型的点状热斑，表明该处电池片内部存在电流受阻与局部升温；而在可见光图像的对应区域表面未见明显特征，符合隐裂类缺陷‘红外可见、可见光难以察觉’的典型表现。
测试用例4 输出：
隐裂属于电池片内部结构性缺陷，通常发生在焊带应力或外力冲击后，导致电流通路受阻，形成局部电流不均。其显著特征是：在红外图像中可见细小的点状光斑，并且由于裂纹位于电池片或封装层内部，表面多保持完整，因此可见光图像中难以直接识别。结合当前这组图像进行分析，未发现符合隐裂特征的内部缺陷。


### TypeOne_D5_Diode_Detect
**怎么判断二极管故障啊？**

In [ ]:
def TypeOne_D5_Diode_Detect(label):
    """
    问题：怎么判断二极管故障啊？
    输入：原始 label 字符串
    输出：自然连贯、专业简明的回答
    """
    faults = parse_label(label)
    diode_faults = [f for f in faults if f.get("type") == "二极管故障"]

    # ① 原理分析：先科学解释
    intro = (
        "二极管故障是光伏组件常见的电气类异常之一。"
        "在正常情况下，旁路二极管可防止被遮挡或损坏的电池片产生反向电流。"
        "当二极管发生开路或短路失效时，电流路径异常，导致局部电池片或整串功率不均，"
        "从而出现典型的热斑效应。"
        "在红外图像上，二极管故障通常表现为沿电路方向分布的条带状或整列发热区域，"
        "温度明显高于周围；而在可见光图像上，组件表面多无直观特征，外观正常，"
        "因为该问题源于接线盒或内部电路，不会直接影响外层玻璃。"
    )

    # ② 结合该组图像
    if not diode_faults:
        return (
            intro
            + " 结合当前这组图像，未发现符合二极管故障特征的异常："
            "红外图像中未出现沿电路方向连续发热的条带状热斑，"
            "可见光图像表面也无相关异常，组件工作状态正常。"
        )

    # 若存在二极管故障
    ir_locs = "、".join([f["ir_coords"] for f in diode_faults if f.get("ir_coords")])
    return (
        intro
        + f" 在当前这组图像中，红外图像于{ir_locs}区域可见典型条带状热斑，"
        "表明该处旁路二极管可能存在短路或开路失效；"
        "而在可见光图像的对应位置表面未见可辨识特征，"
        "符合二极管故障‘红外明显、可见光无变化’的典型表现。"
    )
print("测试用例1 输出：")
print(TypeOne_D5_Diode_Detect(label1))
print("\n测试用例2 输出：")
print(TypeOne_D5_Diode_Detect(label2))
print("\n测试用例3 输出：")
print(TypeOne_D5_Diode_Detect(label3))
print("测试用例4 输出：")
print(TypeOne_D5_Diode_Detect(label4))

测试用例1 输出：
二极管故障是光伏组件常见的电气类异常之一。在正常情况下，旁路二极管可防止被遮挡或损坏的电池片产生反向电流。当二极管发生开路或短路失效时，电流路径异常，导致局部电池片或整串功率不均，从而出现典型的热斑效应。在红外图像上，二极管故障通常表现为沿电路方向分布的条带状或整列发热区域，温度明显高于周围；而在可见光图像上，组件表面多无直观特征，外观正常，因为该问题源于接线盒或内部电路，不会直接影响外层玻璃。 结合当前这组图像，未发现符合二极管故障特征的异常：红外图像中未出现沿电路方向连续发热的条带状热斑，可见光图像表面也无相关异常，组件工作状态正常。

测试用例2 输出：
二极管故障是光伏组件常见的电气类异常之一。在正常情况下，旁路二极管可防止被遮挡或损坏的电池片产生反向电流。当二极管发生开路或短路失效时，电流路径异常，导致局部电池片或整串功率不均，从而出现典型的热斑效应。在红外图像上，二极管故障通常表现为沿电路方向分布的条带状或整列发热区域，温度明显高于周围；而在可见光图像上，组件表面多无直观特征，外观正常，因为该问题源于接线盒或内部电路，不会直接影响外层玻璃。 在当前这组图像中，红外图像于坐标(557, 155)到(587, 165)区域可见典型条带状热斑，表明该处旁路二极管可能存在短路或开路失效；而在可见光图像的对应位置表面未见可辨识特征，符合二极管故障‘红外明显、可见光无变化’的典型表现。

测试用例3 输出：
二极管故障是光伏组件常见的电气类异常之一。在正常情况下，旁路二极管可防止被遮挡或损坏的电池片产生反向电流。当二极管发生开路或短路失效时，电流路径异常，导致局部电池片或整串功率不均，从而出现典型的热斑效应。在红外图像上，二极管故障通常表现为沿电路方向分布的条带状或整列发热区域，温度明显高于周围；而在可见光图像上，组件表面多无直观特征，外观正常，因为该问题源于接线盒或内部电路，不会直接影响外层玻璃。 结合当前这组图像，未发现符合二极管故障特征的异常：红外图像中未出现沿电路方向连续发热的条带状热斑，可见光图像表面也无相关异常，组件工作状态正常。
测试用例4 输出：
二极管故障是光伏组件常见的电气类异常之一。在正常情况下，旁路二极管可防止被遮挡或损坏的电池片产生反向电流。当二极管发生开路或短路失效时，电流路径异常，导致局部电池片或整串功率不均，从而出

### TypeOne_D4_Repair_Order
**对发热重点区域温度级别做出分类，给出维修顺序**

In [ ]:
def TypeOne_D4_Repair_Order(label):
    """
    对发热重点区域温度级别做出分类，给出维修顺序
    """
    faults = parse_label(label)
    if not faults:
        return ("对本组红外图像的发热重点区域进行分类与维修分析，未识别到热斑。"
                "组件受光与温度分布均匀，无需维修；建议纳入例行巡检并在高辐照时段抽样复测。")

    # 分级与策略（本题定制）
    D = {
        # 高风险/电气类
        "组串短路":   {"sev":"极高温区", "prio":1, "feat":"连续点状强热斑/显著过热", "cat":"high",
                       "act":"立即断开该组串，逐段排查并修复短路点；完成后以绝缘电阻测试与红外复测确认热斑消除。"},
        "组串开路":   {"sev":"高温区",   "prio":2, "feat":"大范围或整串温升/热斑，整组停发", "cat":"high",
                       "act":"检查熔断器、端子与插头连接并修复开路点；复测组串电压连续性与温度均衡。"},
        "二极管故障": {"sev":"高温区",   "prio":3, "feat":"沿电路方向条带状发热，局部温度显著升高", "cat":"high",
                       "act":"检查接线盒；可独立更换则更换旁路二极管并复测，否则整板更换。"},
        # 中温/结构或会恶化
        "内部缺陷":   {"sev":"中高温区", "prio":4, "feat":"点状热斑/局部升温，属隐性内部异常", "cat":"mid",
                       "act":"使用EL/I-V确认范围与衰减；衰减明显或持续升温时更换组件。"},
        "表面破损":   {"sev":"中温区",   "prio":5, "feat":"局部区域热斑，玻璃破裂有进水风险", "cat":"mid",
                       "act":"及时更换破损组件，防止水汽进入；更换后红外复测。"},
        "组件异位":   {"sev":"中温区",   "prio":6, "feat":"大面积温差/异常反光，受光角度失衡", "cat":"mid",
                       "act":"校正并加固安装角度，复测电压电流平衡与温度分布。"},
        # 低温/表面类
        "草木遮挡":   {"sev":"中低温区", "prio":7, "feat":"不规则局部热斑，枝叶遮挡受光", "cat":"low",
                       "act":"清理枝叶杂草后红外复测；功率恢复则无需更换组件。"},
        "固定设施遮挡":{"sev":"低温区",   "prio":8, "feat":"固定阴影造成轻度温升", "cat":"low",
                       "act":"调整设施位置/角度以消除遮挡，并随后复测观察。"},
        "表面污迹":   {"sev":"低温区",   "prio":9, "feat":"轻微局部热斑（鸟粪/泥土等）", "cat":"low",
                       "act":"清洗组件后红外复测；若仍有温升，再进行I-V抽测。"},
        "边缘积尘":   {"sev":"极低温区", "prio":10,"feat":"边缘带状轻度温升", "cat":"low",
                       "act":"常规保洁时清除并抽样复测，无需停串级大修。"},
    }

    # 聚合同类并保留红外坐标
    from collections import defaultdict
    grouped = defaultdict(list)
    for f in faults:
        t = f.get("type")
        if t in D:
            grouped[t].append(f.get("ir_coords") or "红外坐标缺失")

    # —— 头部：归类统计（不含策略）——
    types_ordered = sorted(grouped.keys(), key=lambda t: D[t]["prio"])
    head = (f"对本组红外图像的发热重点区域进行分类与维修分析，共识别到 {sum(len(v) for v in grouped.values())} 处热斑，"
            f"涉及类型：{'、'.join(types_ordered)}。")

    # 依次简要说明各类的温级与对应区域（先不谈策略）
    bullets = []
    for t in types_ordered:
        d = D[t]
        ir_str = "；".join(grouped[t])
        bullets.append(f"• {t}：温度级别为{d['sev']}，红外重点区域位于 {ir_str}，表现为{d['feat']}。")
    bullets_text = " ".join(bullets)

    # —— 通用处理原则（填入“本次实际出现的类型”）——
    high_present = [t for t in types_ordered if D[t]["cat"] == "high"]
    mid_present  = [t for t in types_ordered if D[t]["cat"] == "mid"]
    low_present  = [t for t in types_ordered if D[t]["cat"] == "low"]

    def join_or_none(arr):
        return "、".join(arr) if arr else "（本次未见）"

    principle = (
        "应首先处理高温、高风险的电气类问题（如组串短路、组串开路、二极管故障），"
        "以消除潜在安全隐患；随后依次处置中温或结构性缺陷（如内部缺陷、表面破损、组件异位），"
        "最后对表面性轻度异常（如草木遮挡、固定设施遮挡、表面污迹、边缘积尘）进行常规维护或清洁。"
        "整个维修过程中应在每一阶段完成后通过红外复测确认热斑消除，"
        "必要时结合I-V或EL测试验证组件性能恢复。"
    )

    # —— 本次具体顺序 ——
    if len(types_ordered) == 1:
        seq_line = f"综合温度等级与风险评估，本次仅涉及「{types_ordered[0]}」，建议优先完成该项缺陷的处理。"
    else:
        seq_line = "综合温度等级与风险评估，建议维修顺序依次为：" + " → ".join(types_ordered) + "。"

    # —— 本次“按顺序的具体策略” ——
    steps = []
    for i, t in enumerate(types_ordered, start=1):
        d = D[t]
        ir_str = "；".join(grouped[t])
        steps.append(f"{i}. {t}（对应红外图像上：{ir_str}）：{d['act']}")
    steps_text = "本次涉及类型的具体处置要点如下： " + " ".join(steps)

    return " ".join([head, bullets_text, principle, seq_line, steps_text])


print("测试用例1 输出：")
print(TypeOne_D4_Repair_Order(label1))
print("\n测试用例2 输出：")
print(TypeOne_D4_Repair_Order(label2))
print("\n测试用例3 输出：")
print(TypeOne_D4_Repair_Order(label3))
print("测试用例4 输出：")
print(TypeOne_D4_Repair_Order(label4))


测试用例1 输出：
对本组红外图像的发热重点区域进行分类与维修分析，共识别到 1 处热斑，涉及类型：表面破损。 • 表面破损：温度级别为中温区，红外重点区域位于 坐标(365, 148)到(421, 176)，表现为局部区域热斑，玻璃破裂有进水风险。 应首先处理高温、高风险的电气类问题（如组串短路、组串开路、二极管故障），以消除潜在安全隐患；随后依次处置中温或结构性缺陷（如内部缺陷、表面破损、组件异位），最后对表面性轻度异常（如草木遮挡、固定设施遮挡、表面污迹、边缘积尘）进行常规维护或清洁。整个维修过程中应在每一阶段完成后通过红外复测确认热斑消除，必要时结合I-V或EL测试验证组件性能恢复。 综合温度等级与风险评估，本次仅涉及「表面破损」，建议优先完成该项缺陷的处理。 本次涉及类型的具体处置要点如下： 1. 表面破损（对应红外图像上：坐标(365, 148)到(421, 176)）：及时更换破损组件，防止水汽进入；更换后红外复测。

测试用例2 输出：
对本组红外图像的发热重点区域进行分类与维修分析，共识别到 3 处热斑，涉及类型：组串开路、草木遮挡、边缘积尘。 • 组串开路：温度级别为高温区，红外重点区域位于 坐标(557, 155)到(587, 165)，表现为大范围或整串温升/热斑，整组停发。 • 草木遮挡：温度级别为中低温区，红外重点区域位于 坐标(73, 81)到(110, 108)，表现为不规则局部热斑，枝叶遮挡受光。 • 边缘积尘：温度级别为极低温区，红外重点区域位于 坐标(200, 300)到(250, 350)，表现为边缘带状轻度温升。 应首先处理高温、高风险的电气类问题（如组串短路、组串开路、二极管故障），以消除潜在安全隐患；随后依次处置中温或结构性缺陷（如内部缺陷、表面破损、组件异位），最后对表面性轻度异常（如草木遮挡、固定设施遮挡、表面污迹、边缘积尘）进行常规维护或清洁。整个维修过程中应在每一阶段完成后通过红外复测确认热斑消除，必要时结合I-V或EL测试验证组件性能恢复。 综合温度等级与风险评估，建议维修顺序依次为：组串开路 → 草木遮挡 → 边缘积尘。 本次涉及类型的具体处置要点如下： 1. 组串开路（对应红外图像上：坐标(557, 155)到(587, 165)）：检查熔断器、端子与插头连接并修复开路点；复测组串电压连续性与温度均衡。 2

### TypeOne_D4_UAV_Nav
**知道这个光伏板坏了能不能直接无人机导航过去？**

In [ ]:
# 针对“知道这个光伏板坏了能不能直接无人机导航过去？”的行动导向描述
# 先说明无人机“导航过去要做什么 & 有什么用”，再结合当前label里的缺陷与坐标，给出到点要执行的具体动作。
UAV_ACTIONS_FOR_Q = {
    "组串短路": "热像精细复测并绘制温差剖面，标注发热点坐标与组件编号，为地面断电与线缆排查提供准确落点；同步拍RGB确认接头/线缆可见异常。",
    "组串开路": "沿组串做红外带状复飞，核实温升分布与断点大致范围；拍RGB记录连接端子与汇流箱区域，便于后续电压连续性测试就位。",
    "二极管故障": "对接线盒区域做热像条带确认，记录最高温点与时间稳定性；拍RGB特写接线盒外观与铭牌，形成更换取证材料。",
    "内部缺陷": "在热像上做微热斑复测与稳定性观察，锁定单块组件与子串位置；拍RGB记录组件ID/行列，便于后续EL/I-V离线检测对位。",
    "表面破损": "拍RGB高分辨率裂纹/缺角特写与整体视角，量化破损范围；热像复测观察是否持续升温，为更换前后的对比提供基线。",
    "组件异位": "拍RGB近距姿态与支架连接细节，校核错位角度；配合热像做大面温差快照，用于复装校正验收。",
    "草木遮挡": "拍RGB量化遮挡范围与遮挡源位置，标注清理点位；清理后按同航线复飞热像复测作业前后对比。",
    "固定设施遮挡": "拍RGB记录遮挡物与阵列相对位置，输出角度/位置优化参考；热像快照验证遮挡引起的温差带。",
    "表面污迹": "拍RGB记录污渍分布密度与覆盖比例，生成清洗工单落点；清洗后复飞热像验证温升是否消除。",
    "边缘积尘": "拍RGB记录边缘条带污染长度与一致性，归档为批量清洁依据；抽样热像快照校验治理效果。"
}

def TypeOne_D4_UAV_Nav(label, actions=UAV_ACTIONS_FOR_Q):
    """
    问题：知道这个光伏板坏了能不能直接无人机导航过去？
    逻辑：
      1) 先说明“无人机导航过去要做什么、有什么用”（通用说明）；
      2) 再结合本组图像里每类缺陷的红外坐标，说明无人机到点后要做的具体动作。
    """
    faults = parse_label(label)
    if not faults:
        return (
            "可以。无人机导航过去的价值在于：按坐标快速复飞，做红外精细复测、RGB近距取证、"
            "组件/行列标注与工单落点确认，并能在整改后按同航线复飞形成前后对比。"
            "当前这组图像未标出热斑或故障点，如需导航可执行例行抽检航线与基线记录。"
        )

    # 1) 通用说明：导航去“干什么、有什么用”
    intro = (
        "可以。无人机导航过去的作用主要有：按热斑坐标快速到点复飞，"
        "进行红外精细复测与温差量化、RGB近距取证与组件/行列标注，"
        "为后续地面检修（断电、排线、清理、更换）提供准确落点；"
        "整改后可按同一航线复飞，形成前后对比证据闭环。"
    )

    # 2) 结合本组图像：分类型合并坐标，说明到点要做的动作
    from collections import defaultdict
    grouped = defaultdict(list)
    for f in faults:
        grouped[f.get("type", "未定义")].append(f.get("ir_coords") or "红外坐标缺失")

    type_order = list(grouped.keys())
    parts = [intro, f"结合当前图像，已识别到{len(type_order)}类待复飞点位："]
    for t in type_order:
        coords = "；".join(grouped[t])
        action = actions.get(t, "执行热像复测与RGB取证，标注组件与行列位置。")
        parts.append(f"— {t}：红外位置 {coords}。导航到点后，{action}")

    # 3) 收束一句（不做风险评述，聚焦“能做什么/怎么做”）
    parts.append("综上，建议按上述坐标依次导航到点，完成热像复测、RGB取证与标注；整改后按同航线复飞完成对比验证。")
    return " ".join(parts)


print("测试用例1 输出：")
print(TypeOne_D4_UAV_Nav(label1))
print("\n测试用例2 输出：")
print(TypeOne_D4_UAV_Nav(label2))
print("\n测试用例3 输出：")
print(TypeOne_D4_UAV_Nav(label3))
print("测试用例4 输出：")
print(TypeOne_D4_UAV_Nav(label4))

测试用例1 输出：
可以。无人机导航过去的作用主要有：按热斑坐标快速到点复飞，进行红外精细复测与温差量化、RGB近距取证与组件/行列标注，为后续地面检修（断电、排线、清理、更换）提供准确落点；整改后可按同一航线复飞，形成前后对比证据闭环。 结合当前图像，已识别到1类待复飞点位： — 表面破损：红外位置 坐标(365, 148)到(421, 176)。导航到点后，拍RGB高分辨率裂纹/缺角特写与整体视角，量化破损范围；热像复测观察是否持续升温，为更换前后的对比提供基线。 综上，建议按上述坐标依次导航到点，完成热像复测、RGB取证与标注；整改后按同航线复飞完成对比验证。

测试用例2 输出：
可以。无人机导航过去的作用主要有：按热斑坐标快速到点复飞，进行红外精细复测与温差量化、RGB近距取证与组件/行列标注，为后续地面检修（断电、排线、清理、更换）提供准确落点；整改后可按同一航线复飞，形成前后对比证据闭环。 结合当前图像，已识别到3类待复飞点位： — 草木遮挡：红外位置 坐标(73, 81)到(110, 108)。导航到点后，拍RGB量化遮挡范围与遮挡源位置，标注清理点位；清理后按同航线复飞热像复测作业前后对比。 — 边缘积尘：红外位置 坐标(200, 300)到(250, 350)。导航到点后，拍RGB记录边缘条带污染长度与一致性，归档为批量清洁依据；抽样热像快照校验治理效果。 — 组串开路：红外位置 坐标(557, 155)到(587, 165)。导航到点后，沿组串做红外带状复飞，核实温升分布与断点大致范围；拍RGB记录连接端子与汇流箱区域，便于后续电压连续性测试就位。 综上，建议按上述坐标依次导航到点，完成热像复测、RGB取证与标注；整改后按同航线复飞完成对比验证。

测试用例3 输出：
可以。无人机导航过去的作用主要有：按热斑坐标快速到点复飞，进行红外精细复测与温差量化、RGB近距取证与组件/行列标注，为后续地面检修（断电、排线、清理、更换）提供准确落点；整改后可按同一航线复飞，形成前后对比证据闭环。 结合当前图像，已识别到4类待复飞点位： — 表面破损：红外位置 坐标(365, 148)到(421, 176)。导航到点后，拍RGB高分辨率裂纹/缺角特写与整体视角，量化破损范围；热像复测观察是否持续升温，为更换前后的对比提供基线。 — 草木遮挡：红外位置 坐标(

### TypeOne_D7_Hotpot_cause
**光伏板上有热斑是什么造成的**

In [ ]:
def TypeOne_D7_Hotpot_cause(label):
    """
    光伏板上有热斑是什么造成的？
    """
    # 热斑的科学解释
    intro = (
    "热斑是指光伏板中因部分电池片被遮挡、损坏或连接不良，导致其在光照下不能正常工作，反而成为负载，消耗其他正常电池片产生的电能，从而局部发热的现象。"
    "该现象可能引发组件效率下降、材料老化甚至起火。"
    )

    faults = parse_label(label)
    if not faults:
        return intro + (
            "对这组图像进行分析，未发现明显热斑或局部温升，"
            "说明组件受光均匀、工作状态正常。"
        )

    # 定义每类缺陷的典型成因
    desc = {
        "组串短路": "电缆或端子绝缘层破损、接头进水或导线磨损导致正负极直接短接，使局部电流剧增并形成强热斑。",
        "组串开路": "线路断裂或接头松动使电流中断，导致周边组件功率输出不均形成高温区。",
        "二极管故障": "旁路二极管失效或过载烧毁，电流无法分流，受影响区域出现条带状过热。",
        "内部缺陷": "电池片隐裂、断栅、虚焊等内部电气不连续导致载流路径阻塞，引起点状发热。",
        "表面破损": "玻璃破裂或背板损伤使内部电路受潮，局部电阻增加，形成区域热斑。",
        "组件异位": "安装角度偏移导致局部光照强度异常，功率不平衡引发面状热斑。",
        "草木遮挡": "叶片或枝条遮挡部分电池片，使受光不均，电流反向流动在遮挡边缘形成中温热斑。",
        "固定设施遮挡": "支架、栏杆等遮挡长期存在，局部光照不足形成稳定热斑。",
        "表面污迹": "鸟粪、灰尘等局部污染遮挡入射光，造成小范围温升。",
        "边缘积尘": "组件边缘堆积灰尘，透光率下降，形成带状轻度热斑。"
    }

    from collections import defaultdict
    grouped = defaultdict(list)
    for f in faults:
        if f["type"] in desc:
            grouped[f["type"]].append(f.get("ir_coords") or "红外坐标缺失")

    # 主体回答
    parts = [
        "热斑是由于局部受光不均或电流通路异常，造成电池片反向偏置并局部发热的现象。",
        "对这组图像进行分析，"
        f"共识别到{sum(len(v) for v in grouped.values())}处热斑，"
        f"涉及类型有{'、'.join(grouped.keys())}。"
    ]
    for t, locs in grouped.items():
        parts.append(
            f"在红外图像的 {'；'.join(locs)} 区域，可见与“{t}”相符的热斑，主要由{desc[t]}"
        )

    # 总结
    summary = (
        "总体来看，热斑的根本成因是局部光伏电池片无法正常发电，反而作为负载消耗电能，导致局部发热，而上述缺陷均通过不同机制（如遮挡、断路、短路、内部损伤、污染等）触发这一过程。"
        # "可进一步明确其具体原因并指导维修。"
    )

    return " ".join([intro, "".join(parts), summary])

print("测试用例1 输出：")
print(TypeOne_D7_Hotpot_cause(label1))
print("\n测试用例2 输出：")
print(TypeOne_D7_Hotpot_cause(label2))
print("\n测试用例3 输出：")
print(TypeOne_D7_Hotpot_cause(label3))
print("测试用例4 输出：")
print(TypeOne_D7_Hotpot_cause(label4))

测试用例1 输出：
热斑是指光伏板中因部分电池片被遮挡、损坏或连接不良，导致其在光照下不能正常工作，反而成为负载，消耗其他正常电池片产生的电能，从而局部发热的现象。该现象可能引发组件效率下降、材料老化甚至起火。 热斑是由于局部受光不均或电流通路异常，造成电池片反向偏置并局部发热的现象。对这组图像进行分析，共识别到1处热斑，涉及类型有表面破损。在红外图像的 坐标(365, 148)到(421, 176) 区域，可见与“表面破损”相符的热斑，主要由玻璃破裂或背板损伤使内部电路受潮，局部电阻增加，形成区域热斑。 总体来看，热斑的根本成因是局部光伏电池片无法正常发电，反而作为负载消耗电能，导致局部发热，而上述缺陷均通过不同机制（如遮挡、断路、短路、内部损伤、污染等）触发这一过程。

测试用例2 输出：
热斑是指光伏板中因部分电池片被遮挡、损坏或连接不良，导致其在光照下不能正常工作，反而成为负载，消耗其他正常电池片产生的电能，从而局部发热的现象。该现象可能引发组件效率下降、材料老化甚至起火。 热斑是由于局部受光不均或电流通路异常，造成电池片反向偏置并局部发热的现象。对这组图像进行分析，共识别到4处热斑，涉及类型有草木遮挡、边缘积尘、组串开路、二极管故障。在红外图像的 坐标(73, 81)到(110, 108) 区域，可见与“草木遮挡”相符的热斑，主要由叶片或枝条遮挡部分电池片，使受光不均，电流反向流动在遮挡边缘形成中温热斑。在红外图像的 坐标(200, 300)到(250, 350) 区域，可见与“边缘积尘”相符的热斑，主要由组件边缘堆积灰尘，透光率下降，形成带状轻度热斑。在红外图像的 坐标(557, 155)到(587, 165) 区域，可见与“组串开路”相符的热斑，主要由线路断裂或接头松动使电流中断，导致周边组件功率输出不均形成高温区。在红外图像的 坐标(557, 155)到(587, 165) 区域，可见与“二极管故障”相符的热斑，主要由旁路二极管失效或过载烧毁，电流无法分流，受影响区域出现条带状过热。 总体来看，热斑的根本成因是局部光伏电池片无法正常发电，反而作为负载消耗电能，导致局部发热，而上述缺陷均通过不同机制（如遮挡、断路、短路、内部损伤、污染等）触发这一过程。

测试用例3 输出：
热斑是指光伏板中因部分电池片被遮挡、损坏或连接不良，导致其在光照下不能正常工作

### TypeOne_D4_IR_Measure
**热斑这么明显，得带红外仪测温度吧？**